# Exporting excel files into DataFrames

In [ ]:
import numpy as np
np.random.seed(42)

In [ ]:
import pandas as pd
import glob
import numpy as np

directory_path = 'Directory Path'
file_pattern = '*.*'
file_paths = glob.glob(f'{directory_path}/{file_pattern}')
data = {}

for file_path in file_paths:
    file_name = file_path.split('/')[-1]

    try:
        if file_name.endswith('.xlsx'):
            print(file_name)
            df = pd.read_excel(file_path)
        elif file_name.endswith('.csv'):
            print(file_name)
            df = pd.read_csv(file_path)
        else:
            # Handle other file types if needed
            print(f"Ignoring file {file_name} as it has an unsupported extension.")
            continue

        data[file_name] = df

    except Exception as e:
        print(f"Error reading file {file_name}: {e}")
        continue

# Printing keys and basic information about each DataFrame in the data dictionary
for file_name, df in data.items():
    print(f"File Name: {file_name}")
    print(f"DataFrame Info:\n{df.info()}\n{'='*30}\n")

In [ ]:
def filter_data_by_semester(df, start_date, end_date, date_columns):
    # Convert specified columns to datetime
    df[date_columns] = df[date_columns].apply(pd.to_datetime, errors='coerce', dayfirst=True)

    # Check if all cells in the row are empty
    all_empty_mask = df[date_columns].isnull().all(axis=1)

    # Check if any non-empty cell in date_columns is within the range
    any_within_range_mask = ((df[date_columns].notnull()) &
                         ~(df[date_columns] < start_date) &
                         ~(df[date_columns] > end_date)).any(axis=1)

    # Check if some cells are empty and some are not, but at least one non-empty cell is within the range
    mixed_mask = ((df[date_columns].notnull().any(axis=1)) &  # At least one non-empty cell
              (any_within_range_mask))  # At least one non-empty cell is within the range

    # Combine the masks using logical OR (|)
    mask = all_empty_mask | any_within_range_mask | mixed_mask

    # Filter DataFrame based on the mask
    filtered_df = df[mask].copy()

    # Print the indices and date column values of filtered rows
    print(f"Filtered rows for file {file_name}:")
    for index, row in df[~mask].iterrows():
        print(f"Index: {index}, Date column values: {', '.join(str(row[col]) for col in date_columns if not pd.isnull(row[col]))}")

    return filtered_df

# Define start and end dates for the two semesters
first_semester_start = pd.to_datetime('2020-09-21')
second_semester_start = pd.to_datetime('2021-02-08')
end_sem_HW1 = pd.to_datetime('2021-01-14')
end_sem_HW2 = pd.to_datetime('2021-06-02')
end_sem_recht = pd.to_datetime('2021-06-20')
end_sem_acc = pd.to_datetime('2021-01-06')
end_sem_bi = pd.to_datetime('2021-01-22')
end_sem_mark = pd.to_datetime('2021-06-12')
end_sem_mp = pd.to_datetime('2021-01-28')
end_sem_globale = pd.to_datetime('2021-06-25')
end_sem_bafi = pd.to_datetime('2021-06-08')

# Define the date columns for each Excel file
date_columns = {
    '_1_course_info.xlsx': [],
    '_4_enrollment_info.xlsx': ['ENROLLMENT_DATE'],
    '_7_course_contents_info.xlsx': ['DTCREATED', 'DTMODIFIED', 'START_DATE', 'END_DATE'],
    '_7b_course_contents_position.xlsx' : [],
    '_8_course_contents_hierarchy.xlsx' : [],
     '_9_course_contents_adaptive_release_summary.xlsx' : [],
    '_10_course_contents_adaptive_release_global.xlsx': ['AR_START_DATE','AR_END_DATE'],
    '_13_announcement_info.xlsx': ['DTCREATED', 'DTMODIFIED', 'START_DATE', 'END_DATE'],
    '_14_gradable_items.xlsx': ['DUE_DATE'],
    '_15_attempts_and_grades.xlsx': ['ATTEMPT_START_DATE','ATTEMPT_DATE'],
    '_16_posts_info.xlsx': ['DTCREATED', 'DTMODIFIED'],
    '_17_student_post_interaction.xlsx' : [],
    '_18_student_individual_posts_interaction.xlsx': ['FIRST_VIEW', 'LAST_VIEW'],
    '_19_activity_accumulator_logs.csv': ['TIMESTAMP'],
    '_20_activity_accumulator_sessions.csv': ['FIRST_ACTIVITY', 'LAST_ACTIVITY'],
    '_21_toledo_user_activity_logs.csv': ['TIMESTAMP'],
    'Grades.xlsx' : []
}

# Filter data based on the second semester for each file
for file_name, df in data.items():
    filtered_df_second_semester = filter_data_by_semester(df, second_semester_start, end_sem_HW2, date_columns[file_name])
    data[file_name] = filtered_df_second_semester

In [ ]:
# Print out the first few rows of each DataFrame after filtering
# Filter data based on the second semester
for file_name, df in data.items():
    if file_name in date_columns:
        print(f"Filtered data for {file_name} based on the second semester:")
        print(df.head())  # Print the first few rows of the filtered DataFrame
        print("-" * 50)  # Print a separator line

# Train-Test Split

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Make empty dictionaries for train- and testsets
train_sets = {}
test_sets = {}

# Loop for each DataFrame in the data dictionary
for file_name, df in data.items():
    print(f"File Name: {file_name}")

    try:
        nummer = file_name.split('_')[1].split('.')[0]
    except IndexError:
        print(f"Warning: File {file_name} does not contain a number in its name. Assigning default number.")
        nummer = "Default"

    # Name the train- and testsets on the base of the number
    if nummer == "Default":
        train_name = "train_df_grades"
        test_name = "test_df_grades"
    else:
        train_name = f"train_df{nummer}"
        test_name = f"test_df{nummer}"

    # Create an empty DataFrame with the same columns as the original DataFrame
    empty_df = pd.DataFrame(columns=df.columns)

    # Train-test split for each DataFrame
    if len(df) < 2:
        train_df = empty_df  # Assign the empty DataFrame
        test_df = empty_df   # Assign the empty DataFrame
    else:
        train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

    # Add the train- and testsets to the dictionaries
    train_sets[train_name] = train_df
    test_sets[test_name] = test_df

    print(f"Train set named: {train_name}")
    print(f"Test set named: {test_name}")
    print("="*30)

# Preprocessing

In [ ]:
#_15_attempts_and_grades.xlsx
print(train_sets['train_df15'].info())
train_sets['train_df15']["ATTEMPT_START_DATE"] = pd.to_datetime(train_sets['train_df15']["ATTEMPT_START_DATE"],dayfirst=True)
train_sets['train_df15']["ATTEMPT_DATE"] = pd.to_datetime(train_sets['train_df15']["ATTEMPT_DATE"],dayfirst=True)
train_sets['train_df15']["COURSE_PK1"] = train_sets['train_df15']["COURSE_PK1"].astype(str)
train_sets['train_df15']["CONTENT_PK1"] = train_sets['train_df15']["CONTENT_PK1"].astype(str)
train_sets['train_df15']["USER_PK1"] = train_sets['train_df15']["USER_PK1"].astype(str)
train_sets['train_df15']["GRADEBOOK_COLUMN_PK1"] = train_sets['train_df15']["GRADEBOOK_COLUMN_PK1"].astype(str)
one_hot_encoded2 = pd.get_dummies(train_sets['train_df15']['ASSESSMENT_TYPE'], prefix='ASSESSMENT_TYPE_attemptsandgrades')
train_sets['train_df15'] = pd.concat([train_sets['train_df15'], one_hot_encoded2], axis=1)

In [ ]:
#_16_posts_info.xlsx
print(train_sets['train_df16'].info())
train_sets['train_df16']["DTCREATED_postsinfo"] = pd.to_datetime(train_sets['train_df16']["DTCREATED"],dayfirst=True)
train_sets['train_df16']["DTMODIFIED_postsinfo"] = pd.to_datetime(train_sets['train_df16']["DTMODIFIED"],dayfirst=True)
train_sets['train_df16']['BINNED_MSG_TEXT_LENGTH'] = pd.cut(train_sets['train_df16']['MSG_TEXT_LENGTH'],bins=[-np.inf, 10, 50,100,200,400, np.inf],labels=["<10","10-50","50-100","100-200","200-400",">400"])
train_sets['train_df16']["CONTEXT_PK1"] = train_sets['train_df16']["CONTEXT_PK1"].astype(str)
train_sets['train_df16']["THREAD_PK1"] = train_sets['train_df16']["THREAD_PK1"].astype(str)
train_sets['train_df16']["REPLY_TO_POST_PK1"] = train_sets['train_df16']["REPLY_TO_POST_PK1"].astype(str)
train_sets['train_df16']["POST_PK1"] = train_sets['train_df16']["POST_PK1"].astype(str)
train_sets['train_df16']["USER_PK1"] = train_sets['train_df16']["USER_PK1"].astype(str)

In [ ]:
#_17_student_post_interaction.xlsx
print(train_sets['train_df17'].info())
train_sets['train_df17']["COURSE_PK1"] = train_sets['train_df17']["COURSE_PK1"].astype(str)
train_sets['train_df17']["USER_PK1"] = train_sets['train_df17']["USER_PK1"].astype(str)
train_sets['train_df17']["CONTEXT_PK1"] = train_sets['train_df17']["CONTEXT_PK1"].astype(str)
one_hot_encoded3 = pd.get_dummies(train_sets['train_df17']['ACTIVITYTYPE'], prefix='ACTIVITYTYPE')
train_sets['train_df17'] = pd.concat([train_sets['train_df17'], one_hot_encoded3], axis=1)
train_sets['train_df17']['binned_NUM_READ_POSTS'] = pd.cut(train_sets['train_df17']['NUM_READ_POSTS'],bins=[-np.inf, 5, 10,20,50, np.inf],labels=["<5","5-10","10-20","20-50",">50"])

In [ ]:
#_18_student_individual_posts_interaction.xlsx
print(train_sets['train_df18'].info())
train_sets['train_df18']["FIRST_VIEW"] = pd.to_datetime(train_sets['train_df18']["FIRST_VIEW"],dayfirst=True)
train_sets['train_df18']["LAST_VIEW"] = pd.to_datetime(train_sets['train_df18']["LAST_VIEW"],dayfirst=True)
train_sets['train_df18']["COURSE_PK1"] = train_sets['train_df18']["COURSE_PK1"].astype(str)
train_sets['train_df18']["CONTEXT_PK1"] = train_sets['train_df18']["CONTEXT_PK1"].astype(str)
train_sets['train_df18']["MESSAGE_PK1"] = train_sets['train_df18']["MESSAGE_PK1"].astype(str)
train_sets['train_df18']["USER_PK1"] = train_sets['train_df18']["USER_PK1"].astype(str)
one_hot_encoded4 = pd.get_dummies(train_sets['train_df18']['ACTIVITY_TYPE'], prefix='ACTIVITY_TYPE')
train_sets['train_df18'] = pd.concat([train_sets['train_df18'], one_hot_encoded4], axis=1)

In [ ]:
#_19_activity_accumulator_logs.csv
#Right datatype
print(train_sets['train_df19'].info())
train_sets['train_df19']["TIMESTAMP_activity_accumulator_logs"] = pd.to_datetime(train_sets['train_df19']["TIMESTAMP"],dayfirst=True)
train_sets['train_df19']["COURSE_PK1"] = train_sets['train_df19']["COURSE_PK1"].astype(str)
train_sets['train_df19']["USER_PK1"] = train_sets['train_df19']["USER_PK1"].astype(str)
train_sets['train_df19']["CONTENT_PK1"] = train_sets['train_df19']["CONTENT_PK1"].astype(str)
train_sets['train_df19']["SESSION_ID"] = train_sets['train_df19']["SESSION_ID"].astype(str)
# What to do with the O column

In [ ]:
#_21_toledo_user_activity_logs.csv
print(train_sets['train_df21'].info())
train_sets['train_df21']["TIMESTAMP_toledo_user_activity_logs"] = pd.to_datetime(train_sets['train_df21']["TIMESTAMP"], dayfirst=True)
train_sets['train_df21']["COURSE_PK1"] = train_sets['train_df21']["COURSE_PK1"].astype(str)
train_sets['train_df21']["USER_PK1"] = train_sets['train_df21']["USER_PK1"].astype(str)
train_sets['train_df21']["CONTENT_PK1"] = train_sets['train_df21']["CONTENT_PK1"].astype(str)
one_hot_encoded6 = pd.get_dummies(train_sets['train_df21']['EVENT'], prefix='EVENT')
train_sets['train_df21'] = pd.concat([train_sets['train_df21'], one_hot_encoded6], axis=1)

In [ ]:
# Preprocessing Y-variabele
workgrades = train_sets['train_df_grades'][['USER_PK1','OPO_ID','Score januari','Score juni']]

workgrades['Score juni'] = workgrades['Score juni'].replace({'': 0, '#': 0})
workgrades['Score juni'] = workgrades['Score juni'].replace([np.nan, np.inf, -np.inf], 0)
workgrades['Score juni'] = pd.to_numeric(workgrades['Score juni'])

bins = [0,7,9,15,20]
labels = ['fail (0-7)','deliberation (8-9)','pass (10-15)','very good (16-20)']
workgrades['Score_bins'] = pd.cut(workgrades['Score juni'],bins=bins,labels=labels,include_lowest=True)


print((workgrades['Score_bins'] == 'fail (0-7)').min())
print(workgrades.info())
print(workgrades.head())
workgrades['Score_bins'] = workgrades['Score_bins'].astype('category')
workgrades_encoded = pd.get_dummies(workgrades, columns=['Score_bins'])
print(workgrades_encoded)
workgrades= pd.merge(workgrades, workgrades_encoded, on=['USER_PK1','Score januari','Score juni','OPO_ID'])
workgrades['USER_PK1'] = workgrades['USER_PK1'].astype(str)
print(workgrades)

# **Feature Engineering**

## **Time Management and Regularity Features**

Paper 1: "Profiling students' self-regulation with learning analytics: a proof of concept", by Liz-Dominguez et al. (2022)

Paper 2: "Predictive power of regularity of pre-class activities in a flipped classroom" by Jovanovic et al. (2019)

### Feature N°1: PERCENTAGE_WEEKS_ACTIVITY_ABOVE_MEDIAN
"The relative amount of weeks in which the activity of the user was higher than the median of all students’ activities that week. With activity each event in file 19"

In [ ]:
time19 = train_sets['train_df19']['TIMESTAMP_activity_accumulator_logs']
train_sets['train_df19']['week_of_year'] = time19.dt.isocalendar().week
work19 = train_sets['train_df19'][['USER_PK1','week_of_year']]
weeklygrouped = work19.groupby(['USER_PK1','week_of_year']).size().reset_index(name='user_count')

Boxplots representing the user count for each "week_of_year" value representing the active students that week

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.boxplot(x='week_of_year',y='user_count',data=weeklygrouped)

A way to measure procrastination of students

In [ ]:
# Median of medians of all weeks
if 'user_count_weekly_median' not in weeklygrouped.columns:
    median_per_week_train = weeklygrouped.groupby('week_of_year')['user_count'].median().reset_index()
    weeklygrouped = weeklygrouped.merge(median_per_week_train, on='week_of_year', suffixes=('','_weekly_median'))

print(weeklygrouped.info())
print(weeklygrouped.head())

# Was the activity of user x in week y higher than the median activity of all users that week?
weeklygrouped['above_median'] = weeklygrouped['user_count'].values > weeklygrouped['user_count_weekly_median'].values
print(weeklygrouped.head())

# How many weeks in total were user x's activity above the weekly median?
above_median_count = weeklygrouped.groupby('USER_PK1')['above_median'].sum().reset_index()
above_median_count.columns = ['USER_PK1','above_median_count']
print(above_median_count.head())

num_weeks = weeklygrouped['week_of_year'].nunique()
print(num_weeks)

above_median_count['percentageweeks_above_median'] = (above_median_count['above_median_count']/num_weeks)*100
print(above_median_count.head(20))
print(above_median_count.info()) 

# Merging the data with the students' results
dfG = workgrades
merge = pd.merge(above_median_count, dfG, on = 'USER_PK1', how = 'inner')
work = merge[['USER_PK1','above_median_count','percentageweeks_above_median','Score_bins']]

### Feature N°2: DAYS_UNTIL_FIRST_LOGIN
"Number of days it took each student to login on the course page for the first time. Taking the start of the semester as a starting point"

In [ ]:
# Weekly activity (number of events) by students
time19 = train_sets['train_df19']
print(time19.head())
grouped_data = time19.groupby('USER_PK1')

start = pd.to_datetime('2021-02-08')
end_sem2 = pd.to_datetime('2021-06-27')

In [ ]:
first_login = grouped_data['TIMESTAMP_activity_accumulator_logs'].min()
days_until_first_login = (first_login - start).dt.days
print(days_until_first_login)

# Merging the data with the students' results
dfG = workgrades
merged = dfG.merge(days_until_first_login.reset_index(name='days_until_first_login'), on='USER_PK1', how='inner')
workingdata_FT2 = merged[['USER_PK1','days_until_first_login']]
print(workingdata_FT2.head())

### Feature N°3: ENTROPY_WEEKLY_SESSION_COUNTS
"Entropy of weekly session counts per student. Once again, the sessions are here the events as in file 19, with no operations performed on these"


In [ ]:
weekly_session_counts_per_student = time19.groupby(['USER_PK1', 'week_of_year']).size().unstack(fill_value=0)

In [ ]:
import numpy as np
from scipy.stats import entropy

# Calculate probability distribution for each student
prob_distribution_per_student = weekly_session_counts_per_student.apply(lambda x: x / x.sum(), axis=1)

# Calculate entropy for each student
entropy_per_student = prob_distribution_per_student.apply(lambda x: entropy(x, base=2) if x.sum() > 0 else 0, axis=1)

print(f"Entropy of weekly session counts per student:")
print(entropy_per_student)

dfG = workgrades

# Merging the data with the students' results
merged = dfG.merge(entropy_per_student.reset_index(name='entropy_per_student'), on='USER_PK1', how='inner')
workingdata_FT5 = merged[['USER_PK1','entropy_per_student']]
print(workingdata_FT5.head())

## Regularity features

Paper 1: "Differences by course discipline on student behaviorn persistence, and achievement in online courses of undergraduate general education" by Finnegan et al.

Paper 2: "Predictive power of regularity of pre-class activities in a flipped classroom" by Jovanovic et al.

Paper 3: "How learning analytics can early predict under-achieving students in a blended medical education course" by Saqr et al.

Paper 4: "Students matter the most in learning analytics: The effects of internal and instructional conditions in predicting academic success" by Jovanovic et al.

### Feature N°1: FREQ_CHANGE_ENGAGEMENT_PATTERN
"Frequency of change in a student’s engagement pattern over the days of the week"

In [ ]:
df19 = train_sets['train_df19']
print(df19.info())

session_duration = df19.groupby("SESSION_ID")["TIMESTAMP_activity_accumulator_logs"].agg(['min','max'])
session_duration['duration'] = session_duration['max'] - session_duration['min']
# Converting column to hours
session_duration['duration_hours'] = session_duration['duration'] / pd.Timedelta(hours=1)
# Limiting durations longer than 8 hours to 8 hours
session_duration['duration_hours'] = session_duration['duration_hours'].clip(upper=8)
# Converting duration back to normal format
session_duration['duration'] = pd.to_timedelta(session_duration['duration_hours'], unit='h')
print(session_duration.head())

In [ ]:
import numpy as np
# Creation of DataFrame with daily activity relative to their weekly total for each student
# Extracting week number
df19['week_number'] = df19['TIMESTAMP_activity_accumulator_logs'].dt.isocalendar().week
# Extracting day of the week
df19['day_of_week'] = df19['TIMESTAMP_activity_accumulator_logs'].dt.dayofweek + 1

# Specifying Semester
work19_FR1 = df19[(df19['week_number']>=6) & (df19['week_number']<=25)]
all_weeks = list(range(6,26))

# Grouping the data based on the student, the week number and the day of the week
grouped_data_FR1 = work19_FR1.groupby(['USER_PK1', 'week_number', 'day_of_week'])

# Calculating total sessions per day per week per student
total_sessions_per_day_per_week_per_student = grouped_data_FR1['SESSION_ID'].nunique().unstack(fill_value=0)
print(total_sessions_per_day_per_week_per_student.head(10))

# Calculating total sessions per week per student
total_sessions_per_week_per_student = total_sessions_per_day_per_week_per_student.sum(axis=1)

# Reindexing to ensure all combinations of students and weeks are included, filling missing values with zeros
all_students = work19_FR1['USER_PK1'].unique()
new_index = pd.MultiIndex.from_product([all_students, all_weeks], names=['USER_PK1', 'week_number'])
total_sessions_per_day_per_week_per_student = total_sessions_per_day_per_week_per_student.reindex(new_index, fill_value=0)

# Replacing NaN values in weeks with no activity with 0
total_sessions_per_day_per_week_per_student.fillna(0, inplace=True)

# Recalculating total sessions per week per student after reindexing
total_sessions_per_week_per_student = total_sessions_per_day_per_week_per_student.sum(axis=1)
print(total_sessions_per_week_per_student.head())

# Calculating relative weights per week per student
relative_weights_per_week_per_student = total_sessions_per_day_per_week_per_student.div(total_sessions_per_week_per_student, axis=0)

# Replacing NaN values in weeks with no activity with 0
relative_weights_per_week_per_student.fillna(0,inplace=True)

print(relative_weights_per_week_per_student.head())

# Calculating the difference between consecutive weeks
diff_between_weeks = relative_weights_per_week_per_student.diff(axis=0)

# Squaring each difference
squared_diff = diff_between_weeks ** 2

# Calculating the mean squared difference for each student
mean_squared_diff_per_student = squared_diff.mean(axis=1)

# Calculating the aggregate number over all weeks for each student
aggregate_mean_squared_diff_per_student = mean_squared_diff_per_student.groupby('USER_PK1').sum()

print("Mean squared difference per student between consecutive weeks:")
print(mean_squared_diff_per_student.head())

print("\nAggregate mean squared difference over all weeks for each student:")
print(aggregate_mean_squared_diff_per_student)

# Merging the data with the students' results
df_grades = workgrades
merged_FR1 = df_grades.merge(aggregate_mean_squared_diff_per_student.reset_index(name='aggregate_mean_squared_diff_per_student'), on='USER_PK1', how='inner')
print(merged_FR1.head())

workingdata_FR1 = merged_FR1[['USER_PK1', 'aggregate_mean_squared_diff_per_student']]
print(workingdata_FR1.head())

### Feature N°2: WEEKLY_LOGIN_ENGAGEMENT
"Measuring engagement by looking at login behavior. A student was considered engaged in a certain week when having logged in 3 days or more in that week"

In [ ]:
df19 = train_sets['train_df19']

# 1. Creating a new column to register the active days per week
df19['week_number'] = df19['TIMESTAMP_activity_accumulator_logs'].dt.isocalendar().week
df19['day_of_week'] = df19['TIMESTAMP_activity_accumulator_logs'].dt.dayofweek + 1

# 2. Grouping the data based on the student and the week number
grouped_data_FR2 = df19.groupby(['USER_PK1', 'week_number'])

# 3. Calculating the total active days per week for each student
active_days_per_week = grouped_data_FR2['day_of_week'].nunique()
print(active_days_per_week)

# 4. Assigning a score of 1 or 0 based on whether the total active days per week is >= 3
weekly_scores = (active_days_per_week >= 3).astype(int)
print(weekly_scores)

# 5. Summing the weekly scores to calculate the total score for each student
total_scores_per_student_FR2 = weekly_scores.groupby('USER_PK1').sum().reset_index(name='total_score_FR2')
print(total_scores_per_student_FR2)

# Merging the data with the students' results
df_grades_FR2 = workgrades
merged_FR2 = df_grades_FR2.merge(total_scores_per_student_FR2.reset_index(), on='USER_PK1', how='inner')
print(merged_FR2.head())

workingdata_FR2 = merged_FR2[['USER_PK1', 'total_score_FR2']]
print(workingdata_FR2.head())

### Feature N°3: FORUM_ENGAGEMENT
"Measuring engagement by looking at the forum posts views. A score of one was assigned when a student views the course materials more than a Z-score of mean course views (using -1.96 and +1.96 for a two-sided 95% confidence interval)"

In [ ]:
df18 = train_sets['train_df18']

num_content_items = df18['MESSAGE_PK1'].nunique()
print(num_content_items)

message_avg_views = df18.groupby('MESSAGE_PK1')['NUM_VIEWS'].mean()
df18 = df18.merge(message_avg_views, on='MESSAGE_PK1', suffixes=('','_avg'))
df18['Z_score'] = (df18['NUM_VIEWS'] - df18['NUM_VIEWS_avg']) / df18['NUM_VIEWS'].std()

Z_score_threshold = 1.96
df18['exceptional_view'] = np.where(df18['Z_score'] > Z_score_threshold, 1, 0)

student_scores_FR3 = df18.groupby('USER_PK1')['exceptional_view'].sum().reset_index()
student_scores_FR3['exceptional_view_percentage'] = round((student_scores_FR3['exceptional_view'] / num_content_items) * 100, 2)
print(student_scores_FR3.head(30))

# Merging the data with the students' results
df_grades_FR3 = workgrades
merged_FR3 = df_grades_FR3.merge(student_scores_FR3.reset_index(), on='USER_PK1', how='inner')
print(merged_FR3.head())

workingdata_FR3 = merged_FR3[['USER_PK1', 'exceptional_view']]
print(workingdata_FR3.head())

### Feature N°4: ASSESSMENT_ENGAGEMENT
"Measuring engagement by looking at assessment attempts"

In [ ]:
df15 = train_sets['train_df15']
df15['CONTENT_PK1'].fillna('nan',inplace=True)
df15_filtered_FR4 = df15[df15['CONTENT_PK1'] != 'nan']
print(df15_filtered_FR4.info())

In [ ]:
# Creation of column where each row represents an attempt of a specific USER_PK1 & CONTENT_PK1 combination
df15_filtered_FR4['attempts'] = 1
attempts_counts_FR4 = df15_filtered_FR4.groupby(['USER_PK1','CONTENT_PK1'])['attempts'].size().reset_index(name=('attempts'))
total_attempts_per_student_FR4 = attempts_counts_FR4.groupby('USER_PK1')['attempts'].sum().reset_index(name='total_score')
print(total_attempts_per_student_FR4)

# Looking at each unique value of CONTENT_PK1 and check whether this appears in the students' attempts
content_items = df15['CONTENT_PK1'].unique()
print(len(content_items)) # Aantal content items in totaal

user_scores_FR4 = {}

for content in content_items:
    content_data = df15[df15['CONTENT_PK1'] == content]
    content_data = content_data.merge(attempts_counts_FR4, on=['USER_PK1', 'CONTENT_PK1'], how='left')
    processed_users = set() # Keep track of processed users such that a user-content item combination is not counted twice

    for index, row in content_data.iterrows():
        user = row['USER_PK1']

        if user in processed_users:
            continue # Skipping the following loop if user has already been processed for this content item

        # Does the user already exist in the dictionary?!
        if user in user_scores_FR4:
            user_scores_FR4[user] += 1 if row['attempts'] > 0 else 0
        else:
            user_scores_FR4[user] = 1 if row['attempts'] > 0 else 0

        processed_users.add(user) # Adding the user to the set of processed users

total_scores_FR4 = pd.DataFrame(list(user_scores_FR4.items()), columns=['USER_PK1','total_score'])

print(total_scores_FR4)

# Merging the data with the students' results
df_grades_FR4 = workgrades
merged_FR4 = df_grades_FR4.merge(total_scores_FR4.reset_index(), on='USER_PK1', how='inner')
print(merged_FR4.head())
workingdata_FR4 = merged_FR4[['USER_PK1', 'total_score']]
print(workingdata_FR4.head())

## **INTERACTION FEATURES**

Paper: "Learning at distance: Effects of interaction traces on academic achievement" by Joksimovic et al. (2015)

In [ ]:
df17 = train_sets['train_df17']
grouped_data = df17.groupby('USER_PK1')

### Feature N°1: TOTAL_NUM_POSTS_READ
"Number of forum posts read"

In [ ]:
sum_read_posts = grouped_data['NUM_READ_POSTS'].sum()
print(sum_read_posts)

# Merging the data with the students' results
dfG = workgrades
merged = dfG.merge(sum_read_posts.reset_index(name='sum_read_posts'), on='USER_PK1', how='inner')
workingdata_FI1 = merged[['USER_PK1','sum_read_posts']]
print(workingdata_FI1.head())

 ### Feature N°2: TOTAL_NUM_COMMENTS
 "Number of comments on forum posts"

In [ ]:
sum_comments = grouped_data['NUM_REPLIES_COMMENTS'].sum()
print(sum_comments)

# Merging the data with the students' results
dfG = workgrades
merged = dfG.merge(sum_comments.reset_index(name='sum_comments'), on='USER_PK1', how='inner')
workingdata_FI2 = merged[['USER_PK1','sum_comments']]
print(workingdata_FI2.head())

In [ ]:
df18 = train_sets['train_df18']
grouped_data = df18.groupby('USER_PK1')

### Feature N°3: TOTAL_NUM_VIEWS
"Total number of views on posts"

In [ ]:
sum_views = grouped_data['NUM_VIEWS'].sum()
print(sum_views)

# Merging the data with the students' results
dfG = workgrades
merged = dfG.merge(sum_views.reset_index(name='sum_views'), on='USER_PK1', how='inner')
workingdata_FI3 = merged[['USER_PK1','sum_views']]
print(workingdata_FI3.head())

### Feature N°4: TOTAL_POSTS_INDICATED_READ
"Number of posts indicated as read"



In [ ]:
sum_ind_read = grouped_data['READ_STATE_IND'].sum()
print(grouped_data.head())

# Merging the data with the students' results
dfG = workgrades
merged = dfG.merge(sum_ind_read.reset_index(name='sum_ind_read'), on='USER_PK1', how='inner')
workingdata_FI4 = merged[['USER_PK1','sum_ind_read']]
print(workingdata_FI4.head())

### Feature N°5: MEAN_TEXT_LENGTH_OF_POSTS
"Mean of students’ post length"

In [ ]:
df16 = train_sets['train_df16']
grouped_data = df16.groupby('USER_PK1')

In [ ]:
mean_text_length = grouped_data['MSG_TEXT_LENGTH'].mean()
print(mean_text_length)

# Merging the data with the students' results
dfG = workgrades
merged = dfG.merge(mean_text_length.reset_index(name='mean_text_length'), on='USER_PK1', how='inner')
workingdata_FI5 = merged[['USER_PK1','mean_text_length']]
print(workingdata_FI5.head())

## Second set of features

Paper: "Discovering Unusual Study Patterns Using Anomaly Detection and XAI" by Tiukhova et al. (2024)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import datetime
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import entropy
pd.set_option('display.max_colwidth', None)

import math

In [ ]:
# Filling empty cells
train_sets['train_df20']['COURSE_PK1'] = train_sets['train_df20']['COURSE_PK1'].fillna(0)
train_sets['train_df20']['COURSE_PK1'] = train_sets['train_df20']['COURSE_PK1'].astype(np.int64)

train_sets['train_df16']['USER_PK1'] = train_sets['train_df16']['USER_PK1'].fillna(0).astype(int)

train_sets['train_df_grades']['Score januari'].replace('#', np.nan, inplace=True)
train_sets['train_df_grades']['Score juni'].replace('#', np.nan, inplace=True)

In [ ]:
data_19 = train_sets['train_df19'].copy()

In [ ]:
data_19 = data_19.assign(TIMESTAMP_NEW=pd.to_datetime(data_19['TIMESTAMP'],dayfirst=True))
data_19['WEEK'] = data_19.apply(lambda x: x['TIMESTAMP_NEW'].isocalendar()[1], axis = 1)
data_19['YEAR'] =  data_19.apply(lambda x: x['TIMESTAMP_NEW'].isocalendar()[0], axis = 1)
data_19['WEEK'] = data_19['WEEK'].astype(str)
data_19['YEAR'] = data_19['YEAR'].astype(str)
data_19['WEEK and YEAR'] = 'Week ' + data_19['WEEK'] + ' of ' + data_19['YEAR']
data_19 = data_19.drop(columns = ['Unnamed: 0'], errors = 'ignore')

data_19['CONTENT_PK1'] = data_19['CONTENT_PK1'].fillna(0)
data_19['CONTENT_PK1'] = data_19['CONTENT_PK1'].astype(int)

In [ ]:
train_sets['train_df7']['CONTENT_PK1'] = train_sets['train_df7']['CONTENT_PK1'].astype(str)
train_sets['train_df7']['COURSE_PK1'] = train_sets['train_df7']['COURSE_PK1'].astype(str)
data_19['CONTENT_PK1'] = data_19['CONTENT_PK1'].astype(str)
data_19['COURSE_PK1'] = data_19['COURSE_PK1'].astype(str)

In [ ]:
# we merge activity data with the course content information to get more details on the activity
data_19 = data_19.merge(train_sets['train_df7'][['COURSE_PK1', 'CONTENT_PK1',
         'CONTENT_TYPE', 'TITLE', 'PATH']], on = ['COURSE_PK1', 'CONTENT_PK1'], how = 'left')
data_19['CONTENT_TYPE'] = data_19.apply(lambda x: x['DATA'] if pd.isna(x['CONTENT_TYPE']) else x['CONTENT_TYPE'], axis = 1)

In [ ]:
def new_session_id_func(session_ids, df):
    sessions= []
    for session_id in tqdm(session_ids):
        session = df[df['SESSION_ID'] == session_id]
        session = session.reset_index()
        indices = session[session['TIME_DIFF_SESSION']>= 7200].index
        n = len(indices)-1
        for i in sorted(indices,reverse=True):
            session.loc[:i-1, 'new_session_id'] = f'session_{session_id}_{n}'
            n = n - 1

        session['new_session_id'] = session['new_session_id'].fillna(f'session_{session_id}_{len(indices)}')
        sessions.append(session)
    return pd.concat(sessions)

In [ ]:
def change_sessions(data):
    data = data.sort_values(by = ['SESSION_ID', 'TIMESTAMP_NEW']) # first sort data within a session
    data['TIME_DIFF_SESSION'] = data.groupby(by = ['SESSION_ID'])['TIMESTAMP_NEW'].diff().dt.seconds.fillna(0) #calculate the time difference between learning activities within a session
    sessions_to_divide = set(data[data['TIME_DIFF_SESSION'] > 7200]['SESSION_ID']) #get the sessions where the difference between learning activities is larger than 2h
    imputed_sessions = new_session_id_func(sessions_to_divide, data[data['SESSION_ID'].isin(sessions_to_divide)]) #apply a function that will create a new session id - subsession
    imputed_sessions = imputed_sessions.drop(columns = 'index')
    imputed_sessions = imputed_sessions.sort_values(by = ['SESSION_ID', 'TIMESTAMP_NEW'])
    imputed_sessions = imputed_sessions.reset_index()
    imputed_sessions = imputed_sessions.drop(columns = 'index')

    left_session = data[~data['SESSION_ID'].isin(sessions_to_divide)] #get the rest of sessions that does not need to be separated
    left_session['new_session_id'] = left_session['SESSION_ID'].copy()
    new_data = pd.concat([imputed_sessions, left_session])
    return new_data

In [ ]:
data_19_new = pd.DataFrame()
for course in set(data_19['COURSE_PK1']):
    course_data = data_19[data_19['COURSE_PK1'] == course]
    course_data_new = change_sessions(course_data)
    data_19_new = pd.concat([data_19_new, course_data_new])

    print(course_data)

In [ ]:
# each row in the table_19 represents an event (a unique combination of timestamp and content PK)
data_19_new['EVENTS'] = 1

print(data_19_new)

In [ ]:
# we take only those posts that we have interactions for.
data_16 = train_sets['train_df16'][train_sets['train_df16']['CONTEXT_PK1'].isin(list(set(train_sets['train_df17']['CONTEXT_PK1'])))]
data_16 = data_16.assign(DTCREATED_NEW=pd.to_datetime(data_16['DTCREATED'],dayfirst=True))
data_16['WEEK_CREATED'] = data_16.apply(lambda x: x['DTCREATED_NEW'].isocalendar()[1], axis = 1)
data_16['YEAR'] = data_16.apply(lambda x: x['DTCREATED_NEW'].isocalendar()[0], axis = 1)
data_16['WEEK_CREATED'] = data_16['WEEK_CREATED'].astype(str)
data_16['YEAR'] = data_16['YEAR'].astype(str)
data_16['WEEK and YEAR created'] = 'Week ' + data_16['WEEK_CREATED'] + ' of ' + data_16['YEAR']

In [ ]:
data_17 = train_sets['train_df17'].copy()

In [ ]:
workgrades["USER_PK1"] = workgrades["USER_PK1"].astype(str)
grades = workgrades

# Feature engineering

In [ ]:
# Semester 2
course_start, course_finish, exam_weeks, semester_weeks  = second_semester_start, end_sem2,('Week 21 of 2021','Week 22 of 2021','Week 23 of 2021','Week 24 of 2021','Week 25 of 2021'), ('Week 6 of 2021','Week 7 of 2021','Week 8 of 2021','Week 9 of 2021','Week 10 of 2021','Week 11 of 2021','Week 12 of 2021','Week 13 of 2021','Week 14 of 2021','Week 15 of 2021','Week 16 of 2021','Week 17 of 2021','Week 18 of 2021','Week 19 of 2021','Week 20 of 2021')

print(course_start)
print(exam_weeks)

In [ ]:
course_duration = (course_finish - course_start).days
course_duration_weeks = course_duration/7

print(course_duration_weeks)

In [ ]:
course_data = data_19_new
# Filtering out the data outside the course timespan
course_data = course_data[(course_data['TIMESTAMP_NEW'] > course_start) &(course_data['TIMESTAMP_NEW'] < course_finish)]
course_data['DATE'] = course_data['TIMESTAMP_NEW'].dt.date
course_data['DATE'] = pd.to_datetime(course_data['DATE'])
course_data['PATH'] = course_data['PATH'].fillna('not specified')

print(course_data)

In [ ]:
course_data_posts = data_16
# We only want to take into account the posts of the current academic year
course_data_posts = course_data_posts[(course_data_posts['DTCREATED_NEW']>=course_start) & (course_data_posts['DTCREATED_NEW'] <= course_finish)]
course_data_posts['DATE_CREATED'] = course_data_posts['DTCREATED_NEW'].dt.date

print(course_data_posts)

In [ ]:
course_data_posts_consume = data_17

## Overall level of activity

The feature engineering is based on the paper  J. Jovanovi ́c, M. Saqr, S. Joksimovi ́c, D. Gaˇsevi ́c, Students matter the most in learning
analytics: The effects of internal and instructional conditions in predicting academic
success, Computers & Education 172 (2021) 104251.

### Feature N°1: ZERO_SESSION_COUNT
"Number of sessions with duration of zero"

In [ ]:
# then we need to calculate each session's duration. To do that, we find the max and min timestamp value per session and substract latter from the former
# we do it to filter out very long/short sessions
session_duration = pd.pivot_table(course_data, values='TIMESTAMP_NEW', index=['COURSE_PK1','USER_PK1', 'new_session_id'],aggfunc='max').fillna(0) - pd.pivot_table(course_data, values='TIMESTAMP_NEW', index=['COURSE_PK1','USER_PK1', 'new_session_id'],aggfunc='min').fillna(0)
session_duration = session_duration.reset_index()

session_duration['SECONDS'] = session_duration['TIMESTAMP_NEW'] / np.timedelta64(1, 's')
session_duration['MINUTES'] = session_duration['TIMESTAMP_NEW'] / np.timedelta64(1, 'm')
session_duration['HOURS'] = session_duration['TIMESTAMP_NEW'] / np.timedelta64(1, 'h')

print(session_duration.info())

In [ ]:
# we calculate the number of sessions with no duration - could be quick access to the announcements
session_count_zero_duration = pd.pivot_table(session_duration[session_duration['SECONDS'] == 0], values='new_session_id', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'new_session_id':'SESSION_COUNT'})

print(session_count_zero_duration.head())

In [ ]:
plt.hist(session_count_zero_duration['SESSION_COUNT'], bins= 20)
plt.title('TOTAL ZERO LENGTH SESSION COUNT')

### Feature N°2: NON_ZERO_SESSION_COUNT
"Number of sessions, excluding the ones with duration of zero"

In [ ]:
# We exclude sessions of length 0
course_data_non_zero = course_data.drop(course_data[course_data['new_session_id'].isin(set(session_duration[session_duration['SECONDS'] == 0]['new_session_id']))].index).copy()

### Reducing length of the 8 hours+ sessions to 8 hours

In [ ]:
# We cut sessions with a duration of more than 8 hours to a duration of 8 hours
session_duration['HOURS'] = np.where(session_duration['HOURS'] > 8, 8, session_duration['HOURS'])

# We exclude the sessions of duration of 0
session_duration = session_duration.drop(session_duration[session_duration['SECONDS']  == 0].index) # including sessions with 0 duration can spoil the average
print(session_duration.head())

In [ ]:
session_duration_plt = plt.hist(session_duration['MINUTES'])

In [ ]:
# getting session count - non-zero sessions
session_count = pd.pivot_table(course_data_non_zero, values='new_session_id', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'new_session_id':'SESSION_COUNT'})

print(session_count.head())

In [ ]:
median_number_sessions = int(session_count['SESSION_COUNT'].median())

print(median_number_sessions)

In [ ]:
plt.hist(session_count['SESSION_COUNT'], bins= 20)
plt.title('TOTAL SESSION COUNT')

###  Feature N°3: AVERAGE_ACTIONS_PER_SESSION
"Average number of  clicks / learning actions each session"

In [ ]:
# getting the number of events per timestamp for future filtering
clicks_per_timestamp = pd.pivot_table(course_data_non_zero, values='EVENTS', index=['COURSE_PK1','USER_PK1', 'new_session_id','TIMESTAMP_NEW'],
                     aggfunc='sum').fillna(0).reset_index()

clicks_per_timestamp.sort_values(by = 'EVENTS', ascending = False)

In [ ]:
# replace the rows with #events > 1 by 1
# as Toledo logs all the subfolders of an opened parent folder as a separate event, we replace this multiple events by just 1 event of openening a folder
mask = clicks_per_timestamp['EVENTS'] > 1
column_name = 'EVENTS'
clicks_per_timestamp.loc[mask, column_name] = 1
clicks_per_timestamp.sort_values(by = 'EVENTS', ascending = False)

In [ ]:
# getting the median number of events (learning actions) per sessions of a user
clicks_per_session = pd.pivot_table(clicks_per_timestamp, values='EVENTS', index=['COURSE_PK1','USER_PK1', 'new_session_id'],
                     aggfunc='sum').fillna(0).reset_index()
clicks_per_user = pd.pivot_table(clicks_per_session, values='EVENTS', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='median').fillna(0).reset_index().rename(columns = {'EVENTS':'CLICKS_PER_SESSION_AVG'})

print(clicks_per_session.head())
print(clicks_per_user.head())

In [ ]:
plt.hist(clicks_per_user['CLICKS_PER_SESSION_AVG'], bins= 20)
plt.title('MEDIAN #LEARNING ACTIONS PER SESSION')

### Feature N°4: TOTAL_SESSION_DURATION
"The total duration of all sessions of each student (in seconds)"

In [ ]:
# getting total session length per user (in seconds)
session_duration_user = pd.pivot_table(session_duration, values='SECONDS', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='sum').fillna(0).reset_index().rename(columns = {'SECONDS':'SESSION_DURATION'})

In [ ]:
plt.hist(session_duration_user['SESSION_DURATION'], bins= 20)
plt.title('TOTAL SESSION LENGTH')

### Feature N°5: AVERAGE_SESSION_DURATION
"The median session length of each user"

In [ ]:
# getting average (median) session length per user
session_duration_user_avg = pd.pivot_table(session_duration, values='SECONDS', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='median').fillna(0).reset_index().rename(columns = {'SECONDS':'SESSION_DURATION_AVG'})

In [ ]:
plt.hist(session_duration_user_avg['SESSION_DURATION_AVG'], bins= 20)
plt.title('MEDIAN SESSION LENGTH - SECONDS')

### Feature N°6: PROPORTION_ACTIVE_WEEKS
"The proportion of weeks a user has had an active week - with this being defined as follows; a week is active when the number of active days is higher than the average"

In [ ]:
active_days = pd.pivot_table(course_data, values='DATE', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'DATE':'ACTIVE_DAYS'})

In [ ]:
# getting the number of active days per week per user
active_days_per_week = pd.pivot_table(course_data, values='DATE', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'DATE':'ACTIVE_DAYS'})

print(active_days_per_week.info())

In [ ]:
# getting the average number of active days per week for this course
active_days_avg_course_train = pd.pivot_table(active_days_per_week, values='ACTIVE_DAYS', index=['COURSE_PK1'],
                     aggfunc='mean')
active_days_avg_course_train = active_days_avg_course_train.to_dict()

print(active_days_avg_course_train)

In [ ]:
# leaving only those weeks with #active days higher than average #active days in a week per course
active_days_per_week = active_days_per_week[active_days_per_week.apply(lambda x: x['ACTIVE_DAYS'] >= active_days_avg_course_train['ACTIVE_DAYS'][x['COURSE_PK1']], axis = 1)]

print(active_days_per_week.info())

In [ ]:
# getting a proportion of active weeks
num_active_weeks = pd.pivot_table(active_days_per_week, values='WEEK and YEAR', index=['COURSE_PK1', 'USER_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'WEEK and YEAR':'ACTIVE_WEEKS'})
num_active_weeks['PROPORTION_ACTIVE_WEEKS'] = num_active_weeks['ACTIVE_WEEKS']/course_duration_weeks

print(num_active_weeks.head())

In [ ]:
plt.hist(num_active_weeks['PROPORTION_ACTIVE_WEEKS'], bins= 10)
plt.title('PROPORTION OF ACTIVE WEEKS')

###  Feature N°7: ACTIVE_DAYS_PROPORTION
"By considering the course duration, a calculation is made on the amount of active days of each user"

In [ ]:
# getting a proportion of active days
active_days['ACTIVE_DAYS_PROPORTION'] = active_days['ACTIVE_DAYS']/course_duration

print(active_days.head())

In [ ]:
plt.hist(active_days['ACTIVE_DAYS_PROPORTION'], bins= 20)
plt.title('PROPORTION OF ACTIVE DAYS')

### Feature N°8: AVERAGE_ACTIVE_DAYS_PER_WEEK
"The median number of active days per week for each user"

In [ ]:
# getting a median number of active days per week
active_days_per_week = pd.pivot_table(course_data, values='DATE', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR'],
                     aggfunc=pd.Series.nunique).reset_index()
active_days_per_week_per_user = pd.pivot_table(active_days_per_week, values='DATE', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='median').reset_index().rename(columns = {'DATE':'AVERAGE_ACTIVE_DAYS_PER_WEEK'})

print(active_days_per_week.head())
print(active_days_per_week_per_user.head())

In [ ]:
plt.hist(active_days_per_week_per_user['AVERAGE_ACTIVE_DAYS_PER_WEEK'], bins= 10)
plt.title('MEDIAN OF #ACTIVE DAYS PER WEEK')

### Feature N°9: AVG_TIME_DIFF_BETWEEN_ACTIVE_DAYS
"For each user, the median time distance between two consecutive active days"

In [ ]:
course_data = course_data.sort_values(by = ['COURSE_PK1','USER_PK1', 'TIMESTAMP_NEW'])

In [ ]:
max_day_session = pd.pivot_table(course_data, values='DATE', index=['COURSE_PK1','USER_PK1', 'new_session_id'],
                     aggfunc='max').reset_index().sort_values(by = ['COURSE_PK1','USER_PK1','DATE'])

max_day_session['DIFF'] = max_day_session['DATE'].diff().dt.days
max_day_session['DIFF'] = max_day_session['DIFF'].clip(lower=0)
max_day_session['DIFF'] = max_day_session['DIFF'].fillna(0)

print(max_day_session.head())

We are considering only the date and drop the dubplicates so that we do not affect the difference between days when there are more than 1 active study section.

In [ ]:
max_day_session = max_day_session.drop_duplicates(subset = ['COURSE_PK1','USER_PK1','DATE'], keep = 'first')

In [ ]:
# getting median time distance between two consecutive active days
avg_time_distance_user = pd.pivot_table(max_day_session, values='DIFF', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='median').reset_index().rename(columns = {'DIFF':'AVG_DIFF'})

print(avg_time_distance_user.head())

In [ ]:
plt.hist(avg_time_distance_user['AVG_DIFF'], bins= 20)
plt.title('MEDIAN TIME DISTANCE BETWEEN CONSECUTIVE ACTIVE DAYS')

## Learning Action Specific Level of Activity

### Feature N°1: ACTIVE_DAYS_PROPORTION_FORUM_CONTRIBUTION
"Proportion of active days for forum contribution"

In [ ]:
# getting a proportion of active days for the posts written
active_days_posts_written = pd.pivot_table(course_data_posts, values='DATE_CREATED', index=['COURSE_PK1','USER_PK1'],aggfunc=pd.Series.nunique).reset_index().rename(columns = {'DATE_CREATED':'ACTIVE_DAYS'})
active_days_posts_written['ACTIVE_DAYS_PROPORTION'] = active_days_posts_written['ACTIVE_DAYS']/course_duration

In [ ]:
plt.hist(active_days_posts_written['ACTIVE_DAYS_PROPORTION'], bins= 10)
plt.title('PROPORTION OF ACTIVE DAYS FOR FORUM CONTRIBUTION')

In [ ]:
# getting the number of active days with posts written per week per user
active_days_per_week_posts_written = pd.pivot_table(course_data_posts, values='DATE_CREATED', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR created'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'DATE_CREATED':'ACTIVE_DAYS'})

In [ ]:
# getting the average number of active days per week per course
active_days_avg_posts_written = pd.pivot_table(active_days_per_week_posts_written, values='ACTIVE_DAYS', index=['COURSE_PK1'],
                     aggfunc='mean')
active_days_avg_posts_written_train = active_days_avg_posts_written.to_dict()

In [ ]:
# leaving only those weeks with #active days higher than average #active days in a week per course
active_days_per_week_posts_written_train = active_days_per_week_posts_written[active_days_per_week_posts_written.apply(lambda x: x['ACTIVE_DAYS'] >= active_days_avg_posts_written['ACTIVE_DAYS'][x['COURSE_PK1']], axis = 1)]

In [ ]:
# getting a proportion of active weeks for the writing posts activity type
num_active_weeks_posts_written = pd.pivot_table(active_days_per_week_posts_written, values='WEEK and YEAR created', index=['COURSE_PK1', 'USER_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'WEEK and YEAR created':'ACTIVE_WEEKS'})
num_active_weeks_posts_written['PROPORTION_ACTIVE_WEEKS'] = num_active_weeks_posts_written['ACTIVE_WEEKS']/course_duration_weeks

In [ ]:
plt.hist(num_active_weeks_posts_written['PROPORTION_ACTIVE_WEEKS'], bins= 5)
plt.title('PROPORTION OF ACTIVE WEEKS FOR FORUM CONTRIBUTION')

###  Feature N°2: PROPORTION_POSTS_READ
"The proportion of posts a student has read"

In [ ]:
total_number_posts = pd.pivot_table(course_data_posts, values='POST_PK1', index=['COURSE_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'POST_PK1':'TOTAL'})

print(total_number_posts)

There is an issue of having more posts in the file 16 than in the file 17. A possible reason might be due to some old posts being captured in reading behavior displayed in the file 17. Below the issue is illustrated.

In [ ]:
pd.pivot_table(course_data_posts, values='POST_PK1', index=['COURSE_PK1', 'CONTEXT_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'POST_PK1':'TOTAL'})

In [ ]:
pd.pivot_table(course_data_posts_consume, values='NUM_READ_POSTS', index=['COURSE_PK1', 'CONTEXT_PK1'],
                     aggfunc='max').reset_index()

We see that the maximal number of posts read per context in the file 17 is higher than the total number of unique posts in this context from the file 16.

In [ ]:
pd.pivot_table(course_data_posts, values='POST_PK1', index=['COURSE_PK1', 'CONTEXT_PK1'],
                     aggfunc=pd.Series.nunique).reset_index()

In [ ]:
course_data_posts['COURSE_PK1'] = course_data_posts['COURSE_PK1'].astype(int)
course_data_posts['CONTEXT_PK1'] = course_data_posts['CONTEXT_PK1'].astype(int)

course_data_posts_consume['COURSE_PK1'] =course_data_posts_consume['COURSE_PK1'].astype(int)
course_data_posts_consume['CONTEXT_PK1'] = course_data_posts_consume['CONTEXT_PK1'].astype(int)

In [ ]:
course_data_posts_consume = course_data_posts_consume.merge(pd.pivot_table(course_data_posts, values='POST_PK1', index=['COURSE_PK1', 'CONTEXT_PK1'],
                     aggfunc=pd.Series.nunique).reset_index(), on = ['COURSE_PK1','CONTEXT_PK1'], how = 'left')

In [ ]:
course_data_posts_consume['NEW_NUM_READ_POSTS'] = course_data_posts_consume.apply(lambda x: x['NUM_READ_POSTS'] if x['NUM_READ_POSTS'] < x['POST_PK1'] else x['POST_PK1'], axis = 1)

In [ ]:
proportion_posts_read = pd.pivot_table(course_data_posts_consume, values='NEW_NUM_READ_POSTS', index=['COURSE_PK1', 'USER_PK1'],
                     aggfunc='sum').reset_index()
proportion_posts_read = proportion_posts_read.merge(total_number_posts, on = ['COURSE_PK1'])

In [ ]:
proportion_posts_read['PROPORTION_POSTS_READ'] = proportion_posts_read['NEW_NUM_READ_POSTS']/proportion_posts_read['TOTAL']

In [ ]:
plt.hist(proportion_posts_read['PROPORTION_POSTS_READ'], bins= 5)
plt.title('PROPORTION OF POSTS READ')

### Feature N°3: POSTS_CREATED
"The amount of posts a student has created"

In [ ]:
posts_created = pd.pivot_table(course_data_posts, values='POST_PK1', index=['COURSE_PK1', 'USER_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'POST_PK1':'POSTS_CREATED'})

print(posts_created.head())

In [ ]:
plt.hist(posts_created['POSTS_CREATED'], bins= 15)
plt.title('POSTS_CREATED')

## Overall Regularity of Study

###  Feature N°1: ENTROPY_CLICKS
"Entropy of clicks/learning action counts per session"

In [ ]:
# calculating total number of clicks per student per course
sum_clicks_dict = pd.pivot_table(clicks_per_session, values='EVENTS', index=['COURSE_PK1', 'USER_PK1'],
                     aggfunc='sum').to_dict()

In [ ]:
clicks_per_session['TOTAL_CLICKS'] = clicks_per_session.apply(lambda x: sum_clicks_dict['EVENTS'][(x['COURSE_PK1'], x['USER_PK1'])], axis = 1)

print(clicks_per_session.head())

In [ ]:
clicks_per_session['PROBA'] = clicks_per_session['EVENTS']/clicks_per_session['TOTAL_CLICKS']

print(clicks_per_session.head())

In [ ]:
# Entropy of learning action counts per session (session of length of 0 are excluded)
clicks_entropy = pd.DataFrame(clicks_per_session.groupby(by = ['COURSE_PK1', 'USER_PK1'])['PROBA'].apply(lambda x: entropy(x, base=2))).reset_index()

print(clicks_entropy.head())

In [ ]:
plt.hist(clicks_entropy['PROBA'], bins= 10)

plt.title('ENTROPY OF LEARNING ACTION COUNTS PER SESSION')

### Feature N°2: ENTROPY_SESSION_LENGTH
"Entropy of students’ session lengths"

In [ ]:
session_duration_user_dict = pd.pivot_table(session_duration, values='SECONDS', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='sum').fillna(0).rename(columns = {'SECONDS':'SESSION_DURATION'}).to_dict()
session_duration['TOTAL_DURATION'] = session_duration.apply(lambda x: session_duration_user_dict['SESSION_DURATION'][(x['COURSE_PK1'], x['USER_PK1'])], axis = 1)
session_duration['PROBA'] = session_duration['SECONDS']/session_duration['TOTAL_DURATION']

In [ ]:
# Entropy of session length
duration_entropy = pd.DataFrame(session_duration.groupby(by = ['COURSE_PK1', 'USER_PK1'])['PROBA'].apply(lambda x: entropy(x, base=2))).reset_index()

print(duration_entropy.head())

In [ ]:
plt.hist(duration_entropy['PROBA'], bins= 10)

plt.title('ENTROPY OF SESSION LENGTH')

### Feature N°3: PROPORTION_WEEKS_FIRST_DAY_ACTIVE
"The proportion of weeks for which the students were active on monday"

In [ ]:
course_data['DAY_OF_WEEK'] = course_data.apply(lambda x: x['DATE'].isoweekday(), axis = 1)

In [ ]:
weeks_with_active_first_day = pd.pivot_table(course_data[course_data['DAY_OF_WEEK'] == 1], values='WEEK and YEAR', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'WEEK and YEAR':'ACTIVE_WEEKS'})
weeks_with_active_first_day
weeks_with_active_first_day['PROPORTION_ACTIVE_WEEKS'] = weeks_with_active_first_day['ACTIVE_WEEKS']/course_duration_weeks

print(weeks_with_active_first_day.head())

In [ ]:
plt.hist(weeks_with_active_first_day['PROPORTION_ACTIVE_WEEKS'], bins= 10)

plt.title('PROPORTION OF WEEKS WITH ACTIVE FIRST DAY')

### Feature N°4: PROPORTION_LA_FIRST_DAY_OF_WEEK
"Proportion of learning actions on monday"

In [ ]:
course_data_non_zero['DAY_OF_WEEK'] = course_data_non_zero.apply(lambda x: x['DATE'].isoweekday(), axis = 1)

In [ ]:
clicks_per_timestamp = pd.pivot_table(course_data_non_zero, values='EVENTS', index=['COURSE_PK1','USER_PK1', 'new_session_id','TIMESTAMP_NEW'],
                     aggfunc='sum').fillna(0).reset_index()

clicks_per_timestamp.sort_values(by = 'EVENTS', ascending = False)

In [ ]:
# replace the rows with #events > 1 by 1
# as Toledo logs all the subfolders of an opened parent folder as a separate event, we replace this multiple events by just 1 event of openening a folder
mask = clicks_per_timestamp['EVENTS'] > 1
column_name = 'EVENTS'
clicks_per_timestamp.loc[mask, column_name] = 1
clicks_per_timestamp.sort_values(by = 'EVENTS', ascending = False)

In [ ]:
clicks_per_timestamp['DAY_OF_WEEK'] = clicks_per_timestamp.apply(lambda x: x['TIMESTAMP_NEW'].isoweekday(), axis = 1)

In [ ]:
clicks_per_timestamp['WEEK'] = clicks_per_timestamp.apply(lambda x: x['TIMESTAMP_NEW'].isocalendar()[1], axis = 1)
clicks_per_timestamp['YEAR'] = clicks_per_timestamp.apply(lambda x: x['TIMESTAMP_NEW'].isocalendar()[0], axis = 1)
clicks_per_timestamp['WEEK'] = clicks_per_timestamp['WEEK'].astype(str)
clicks_per_timestamp['YEAR'] = clicks_per_timestamp['YEAR'].astype(str)
clicks_per_timestamp['WEEK and YEAR'] = 'Week ' + clicks_per_timestamp['WEEK'] + ' of ' + clicks_per_timestamp['YEAR']
clicks_per_timestamp['DATE'] = clicks_per_timestamp.apply(lambda x: x['TIMESTAMP_NEW'].date(), axis = 1)

In [ ]:
total_learning_actions = pd.pivot_table(clicks_per_timestamp, values='EVENTS', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR'],
                     aggfunc='sum').fillna(0).reset_index()

print(total_learning_actions.head())

In [ ]:
learning_actions_first_day = pd.pivot_table(clicks_per_timestamp[clicks_per_timestamp['DAY_OF_WEEK'] == 1], values='EVENTS', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR'],
                     aggfunc='sum').fillna(0).reset_index().rename(columns = {'EVENTS':'EVENTS_FIRST_DAY'})

print(learning_actions_first_day.head())

In [ ]:
total_learning_actions = total_learning_actions.merge(learning_actions_first_day, on = ['COURSE_PK1','USER_PK1','WEEK and YEAR'], how = 'left').fillna(0)
total_learning_actions['PROPORTION'] = total_learning_actions['EVENTS_FIRST_DAY']/total_learning_actions['EVENTS']

print(total_learning_actions.head())

In [ ]:
average_proportion_la_first_day = pd.pivot_table(total_learning_actions, values='PROPORTION', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='median').fillna(0).reset_index()

print(average_proportion_la_first_day.head())

In [ ]:
plt.hist(average_proportion_la_first_day['PROPORTION'], bins= 15)

plt.title('PROPORTION OF LA ON THE FIRST DAY')

## Learning Actions Specific Regularity of Study

In [ ]:
n_weeks = len(set(course_data_non_zero['WEEK and YEAR']))

### Feature N°1: ENTROPY_FORUM_CONTRIBUTION_DAILY
"Entropy of daily posts written "

Entropy for the missing cases is set to the maximal possible entropy - missing not at random cases.

In [ ]:
posts_per_day = pd.pivot_table(course_data_posts, values='POST_PK1', index=['COURSE_PK1','USER_PK1', 'DATE_CREATED'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'POST_PK1':'POSTS_PER_DAY'})

print(posts_per_day.head())

In [ ]:
posts_total = pd.pivot_table(course_data_posts, values='POST_PK1', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'POST_PK1':'POSTS_TOTAL'})

print(posts_total.head())

In [ ]:
posts_per_day = posts_per_day.merge(posts_total, how = 'left', on = ['COURSE_PK1', 'USER_PK1'])
posts_per_day['PROBA'] = (posts_per_day['POSTS_PER_DAY']/posts_per_day['POSTS_TOTAL']).fillna(0)

posts_per_day = pd.DataFrame(posts_per_day.groupby(by = ['COURSE_PK1', 'USER_PK1'])['PROBA'].apply(lambda x: entropy(x, base=2))).reset_index().fillna(math.log2(course_duration))

print(posts_per_day.head())

In [ ]:
plt.hist(posts_per_day['PROBA'], bins= 10)

plt.title('ENTROPY OF DAILY POSTS WRITTEN')

### Feature N°2: ENTROPY_FORUM_CONTRIBUTION_WEEKLY
"Entropy of weekly posts written"

In [ ]:
posts_per_week = pd.pivot_table(course_data_posts, values='POST_PK1', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR created'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'POST_PK1':'POSTS_PER_WEEK'})

In [ ]:
posts_per_week = posts_per_week.merge(posts_total, how = 'left', on = ['COURSE_PK1', 'USER_PK1'])
posts_per_week['PROBA'] = (posts_per_week['POSTS_PER_WEEK']/posts_per_week['POSTS_TOTAL']).fillna(0)

posts_per_week = pd.DataFrame(posts_per_week.groupby(by = ['COURSE_PK1', 'USER_PK1'])['PROBA'].apply(lambda x: entropy(x, base=2))).reset_index().fillna(math.log2(n_weeks))

In [ ]:
plt.hist(posts_per_week['PROBA'], bins= 15)

plt.title('ENTROPY OF WEEKLY POSTS WRITTEN')

# MoneyWalks feature engineering

Based on the paper: V. K. Singh, B. Bozkaya, and A. Pentland, “Money walks: implicit mobility behavior
and financial well-being,” PloS one, vol. 10, no. 8, p. e0136628, 2015.

### Feature N°1: DIVERSITY_OVERALL
"A measure that refers  to how evenly sessions are distributed across weeks"

In [ ]:
sessions_per_week = pd.pivot_table(course_data_non_zero, values='new_session_id', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index().rename(columns = {'new_session_id':'SESSIONS_PER_WEEK'})

print(sessions_per_week.head())

In [ ]:
sessions_total = pd.pivot_table(course_data_non_zero, values='new_session_id', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index().rename(columns = {'new_session_id':'SESSIONS_TOTAL'})

print(sessions_total.head())

In [ ]:
weeks_count_total = pd.pivot_table(course_data_non_zero, values='WEEK and YEAR', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index()
weeks_count_total['LOG NUM OF WEEKS'] = weeks_count_total.apply(lambda x: math.log2(x['WEEK and YEAR']), axis = 1)

print(weeks_count_total.head())

In [ ]:
sessions_per_week = sessions_per_week.merge(sessions_total, how = 'left', on = ['COURSE_PK1', 'USER_PK1'])
sessions_per_week['PROBA'] = (sessions_per_week['SESSIONS_PER_WEEK']/sessions_per_week['SESSIONS_TOTAL']).fillna(0)


sessions_per_week_diversity = pd.DataFrame(sessions_per_week.groupby(by = ['COURSE_PK1', 'USER_PK1'])['PROBA'].apply(lambda x: entropy(x, base=2))).reset_index().fillna(math.log2(n_weeks))
sessions_per_week_diversity = sessions_per_week_diversity.merge(weeks_count_total, how = 'left', on = ['COURSE_PK1','USER_PK1'])
sessions_per_week_diversity['DIVERSITY'] =  (sessions_per_week_diversity['PROBA']/sessions_per_week_diversity['LOG NUM OF WEEKS'])

print(sessions_per_week.head())
print(sessions_per_week_diversity.head())

In [ ]:
plt.hist(sessions_per_week_diversity['DIVERSITY'], bins= 15)

plt.title('DIVERSITY OVERALL')

### Feature N°2: DIVERSITY_EXAM_PERIOD
"A measure that refers  to how evenly sessions are distributed across weeks during the exam period"

In [ ]:
sessions_per_week_exam = pd.pivot_table(course_data_non_zero[course_data_non_zero['WEEK and YEAR'].isin(exam_weeks)], values='new_session_id', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index().rename(columns = {'new_session_id':'SESSIONS_PER_WEEK_EXAM'})

print(sessions_per_week_exam.head())

In [ ]:
sessions_total_exam = pd.pivot_table(course_data_non_zero[course_data_non_zero['WEEK and YEAR'].isin(exam_weeks)], values='new_session_id', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index().rename(columns = {'new_session_id':'SESSIONS_TOTAL_EXAM'})

print(sessions_total_exam)

In [ ]:
weeks_count_total_exam = pd.pivot_table(course_data_non_zero[course_data_non_zero['WEEK and YEAR'].isin(exam_weeks)], values='WEEK and YEAR', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index()
weeks_count_total_exam['LOG NUM OF WEEKS'] = weeks_count_total_exam.apply(lambda x: math.log2(x['WEEK and YEAR']), axis = 1)

In [ ]:
sessions_per_week_exam = sessions_per_week_exam.merge(sessions_total_exam, how = 'left', on = ['COURSE_PK1', 'USER_PK1'])
sessions_per_week_exam['PROBA'] = (sessions_per_week_exam['SESSIONS_PER_WEEK_EXAM']/sessions_per_week_exam['SESSIONS_TOTAL_EXAM']).fillna(0)


sessions_per_week_diversity_exam = pd.DataFrame(sessions_per_week_exam.groupby(by = ['COURSE_PK1', 'USER_PK1'])['PROBA'].apply(lambda x: entropy(x, base=2))).reset_index().fillna(math.log2(n_weeks))
sessions_per_week_diversity_exam = sessions_per_week_diversity_exam.merge(weeks_count_total_exam, how = 'left', on = ['COURSE_PK1','USER_PK1'])
sessions_per_week_diversity_exam['DIVERSITY_EXAM_PERIOD'] =  (sessions_per_week_diversity_exam['PROBA']/sessions_per_week_diversity_exam['LOG NUM OF WEEKS'])

In [ ]:
print(sessions_per_week_diversity_exam['DIVERSITY_EXAM_PERIOD'])

## Loyalty

### Feature N°1: LOYALTY_OVERALL
"Proportion of sessions that took place in the top 3 weeks regarding session count. A higher value represents many sessions concentrated in only a few weeks"

In [ ]:
top3_sessions = sessions_per_week.groupby(by = ['COURSE_PK1','USER_PK1']).apply(lambda x: x.nlargest(3, "SESSIONS_PER_WEEK")).reset_index(drop=True)

In [ ]:
sessions_total_in_top3_weeks = pd.pivot_table(top3_sessions, values='SESSIONS_PER_WEEK', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='sum').fillna(0).reset_index().rename(columns = {'SESSIONS_PER_WEEK':'SESSIONS_TOTAL_TOP3'})

In [ ]:
sessions_total_in_top3_weeks = sessions_total_in_top3_weeks.merge(sessions_total, how='left', on = ['COURSE_PK1','USER_PK1'])

In [ ]:
sessions_total_in_top3_weeks['LOYALTY'] = sessions_total_in_top3_weeks['SESSIONS_TOTAL_TOP3']/sessions_total_in_top3_weeks['SESSIONS_TOTAL']

In [ ]:
plt.hist(sessions_total_in_top3_weeks['LOYALTY'], bins= 15)

plt.title('LOYALTY')

### Feature N°2: LOYALTY_EXAM
"Proportion of sessions that took place in the top 3 weeks regarding session count, focusing on exam period"

In [ ]:
exam_sessions = sessions_per_week[sessions_per_week['WEEK and YEAR'].isin(exam_weeks)]

print(exam_sessions.head())

In [ ]:
sessions_total_in_exam_weeks = pd.pivot_table(exam_sessions, values='SESSIONS_PER_WEEK', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='sum').fillna(0).reset_index().rename(columns = {'SESSIONS_PER_WEEK':'SESSIONS_TOTAL_EXAM'})

In [ ]:
sessions_total_in_exam_weeks = sessions_total_in_exam_weeks.merge(sessions_total, how='left', on = ['COURSE_PK1','USER_PK1'])

In [ ]:
sessions_total_in_exam_weeks['LOYALTY'] = sessions_total_in_exam_weeks['SESSIONS_TOTAL_EXAM']/sessions_total_in_exam_weeks['SESSIONS_TOTAL']

In [ ]:
plt.hist(sessions_total_in_exam_weeks['LOYALTY'], bins= 15)

plt.title('LOYALTY EXAM PERIOD')

## Regularity

### Feature N°1: REGULARITY
"Measurement for the level of diversity in students’ behavior over time"

### Diversity semester weeks

In [ ]:
course_data_non_zero_semester = course_data_non_zero[course_data_non_zero['WEEK and YEAR'].isin(semester_weeks)]

In [ ]:
sessions_per_week_semester = pd.pivot_table(course_data_non_zero_semester, values='new_session_id', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index().rename(columns = {'new_session_id':'SESSIONS_PER_WEEK'})

In [ ]:
sessions_total_semester = pd.pivot_table(course_data_non_zero_semester, values='new_session_id', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index().rename(columns = {'new_session_id':'SESSIONS_TOTAL'})

In [ ]:
weeks_count_total_semester = pd.pivot_table(course_data_non_zero_semester, values='WEEK and YEAR', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index()
weeks_count_total_semester['LOG NUM OF WEEKS'] = weeks_count_total_semester.apply(lambda x: math.log2(x['WEEK and YEAR']), axis = 1)

In [ ]:
sessions_per_week_first_month = sessions_per_week_semester.merge(sessions_total_semester, how = 'left', on = ['COURSE_PK1', 'USER_PK1'])
sessions_per_week_first_month['PROBA'] = (sessions_per_week_first_month['SESSIONS_PER_WEEK']/sessions_per_week_first_month['SESSIONS_TOTAL']).fillna(0)


sessions_per_week_first_month_diversity = pd.DataFrame(sessions_per_week_first_month.groupby(by = ['COURSE_PK1', 'USER_PK1'])['PROBA'].apply(lambda x: entropy(x, base=2))).reset_index().fillna(math.log2(len(semester_weeks)))
sessions_per_week_first_month_diversity = sessions_per_week_first_month_diversity.merge(weeks_count_total_semester, how = 'left', on = ['COURSE_PK1','USER_PK1'])
sessions_per_week_first_month_diversity['DIVERSITY_SEMESTER'] =  sessions_per_week_first_month_diversity['PROBA']/sessions_per_week_first_month_diversity['LOG NUM OF WEEKS']

In [ ]:
diversity_for_regularity = sessions_per_week_diversity[['COURSE_PK1','USER_PK1','DIVERSITY']].merge(sessions_per_week_first_month_diversity[['COURSE_PK1','USER_PK1','DIVERSITY_SEMESTER']], how = 'left', on = ['COURSE_PK1','USER_PK1']).fillna(0)

### Loyalty semester weeks

In [ ]:
top3_sessions_semester = sessions_per_week[sessions_per_week['WEEK and YEAR'].isin(semester_weeks)].groupby(by = ['COURSE_PK1','USER_PK1']).apply(lambda x: x.nlargest(3, "SESSIONS_PER_WEEK")).reset_index(drop=True)

In [ ]:
sessions_total_in_top3_weeks_semester = pd.pivot_table(top3_sessions_semester, values='SESSIONS_PER_WEEK', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='sum').fillna(0).reset_index().rename(columns = {'SESSIONS_PER_WEEK':'SESSIONS_TOTAL_TOP3'})

In [ ]:
sessions_total_in_top3_weeks_semester = sessions_total_in_top3_weeks_semester.merge(sessions_total_semester, how='left', on = ['COURSE_PK1','USER_PK1'])

In [ ]:
sessions_total_in_top3_weeks_semester['LOYALTY_SEMESTER'] = sessions_total_in_top3_weeks_semester['SESSIONS_TOTAL_TOP3']/sessions_total_in_top3_weeks_semester['SESSIONS_TOTAL']

In [ ]:
regularity_df = sessions_per_week_diversity[['COURSE_PK1','USER_PK1','DIVERSITY']].merge(sessions_total_in_top3_weeks[['COURSE_PK1','USER_PK1','LOYALTY']])

In [ ]:
regularity_df = regularity_df.merge(diversity_for_regularity[['COURSE_PK1','USER_PK1','DIVERSITY_SEMESTER']], how = 'left', on = ['COURSE_PK1','USER_PK1']).merge(sessions_total_in_top3_weeks_semester[['COURSE_PK1','USER_PK1','LOYALTY_SEMESTER']], how= 'left', on = ['COURSE_PK1','USER_PK1'] ).fillna(0)

In [ ]:
regularity_df['REGULARITY'] = 1- np.sqrt((regularity_df['DIVERSITY_SEMESTER'] - regularity_df['DIVERSITY'] )**2 +  (regularity_df['LOYALTY_SEMESTER'] - regularity_df['LOYALTY'] )**2)/np.sqrt(2)

In [ ]:
plt.hist(regularity_df['REGULARITY'], bins= 15)

plt.title('REGULARITY')

# Final merging

In [ ]:
grades['Score juni'] = grades.apply(lambda x: np.nan if (pd.isnull(x['Score juni']) or not str(x['Score juni']).replace('.', '').isdigit()) else int(str(x['Score juni']).split('.')[0]), axis=1)

In [ ]:
# Merging all data together
course_grades = grades[['USER_PK1', 'Score januari',
                            'Score juni']].merge(session_count_zero_duration.rename(columns =
                            {'SESSION_COUNT':'ZERO_SESSIONS_COUNT'}),
                             how = 'left', on=['USER_PK1'])

course_grades = course_grades.merge(session_count.rename(columns = {'SESSION_COUNT':'NON_ZERO_SESSION_COUNT'}),
                             how = 'left', on=['COURSE_PK1','USER_PK1'])

course_grades = course_grades.merge(session_duration_user.rename(columns = {'SESSION_DURATION':'TOTAL_SESSION_DURATION'}),
                             how = 'left', on=['COURSE_PK1','USER_PK1'])

course_grades = course_grades.merge(session_duration_user_avg.rename(columns = {'SESSION_DURATION_AVG':'AVERAGE_SESSION_DURATION'}),
                             how = 'left', on=['COURSE_PK1','USER_PK1'])

course_grades = course_grades.merge(clicks_per_user.rename(columns = {'CLICKS_PER_SESSION_AVG':'AVERAGE_ACTIONS_PER_SESSION'}),
                             how = 'left', on=['COURSE_PK1','USER_PK1'])

course_grades = course_grades.merge(num_active_weeks[['COURSE_PK1','USER_PK1','PROPORTION_ACTIVE_WEEKS']],
                             how = 'left', on=['COURSE_PK1','USER_PK1'])

course_grades = course_grades.merge(active_days[['COURSE_PK1','USER_PK1','ACTIVE_DAYS_PROPORTION']],
                             how = 'left', on=['COURSE_PK1','USER_PK1'])

course_grades = course_grades.merge(active_days_per_week_per_user, how = 'left', on=['COURSE_PK1','USER_PK1'])

course_grades = course_grades.merge(avg_time_distance_user.rename(columns = {'AVG_DIFF':'AVG_TIME_DIFF_BETWEEN_ACTIVE_DAYS'}),
                             how = 'left', on=['COURSE_PK1','USER_PK1'])


course_grades['COURSE_PK1'] = course_grades['COURSE_PK1'].astype(float).fillna(0)
course_grades['USER_PK1']=course_grades['USER_PK1'].astype(int)
course_grades['COURSE_PK1']=course_grades['COURSE_PK1'].astype(int)


course_grades = course_grades.merge(active_days_posts_written.rename(columns = {'ACTIVE_DAYS_PROPORTION':
                                'ACTIVE_DAYS_PROPORTION_FORUM_CONTRIBUTION'})[['COURSE_PK1','USER_PK1',
                                'ACTIVE_DAYS_PROPORTION_FORUM_CONTRIBUTION']], how = 'left', on=['COURSE_PK1','USER_PK1'])

course_grades = course_grades.merge(num_active_weeks_posts_written.rename(columns = {'PROPORTION_ACTIVE_WEEKS':
                                'PROPORTION_ACTIVE_WEEKS_FORUM_CONTRIBUTION'})[['COURSE_PK1','USER_PK1',
                                'PROPORTION_ACTIVE_WEEKS_FORUM_CONTRIBUTION']], how = 'left', on=['COURSE_PK1','USER_PK1'])
course_grades['USER_PK1']=course_grades['USER_PK1'].astype(int)


proportion_posts_read['USER_PK1'] = proportion_posts_read['USER_PK1'].astype(int)
course_grades = course_grades.merge(proportion_posts_read[['COURSE_PK1','USER_PK1','PROPORTION_POSTS_READ']],
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])

posts_created['USER_PK1']=posts_created['USER_PK1'].astype(int)
posts_created['COURSE_PK1']=posts_created['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(posts_created[['COURSE_PK1','USER_PK1','POSTS_CREATED']],how = 'left', on=['COURSE_PK1','USER_PK1'])

clicks_entropy['USER_PK1']=clicks_entropy['USER_PK1'].astype(int)
clicks_entropy['COURSE_PK1']=clicks_entropy['COURSE_PK1'].astype(int)


course_grades = course_grades.merge(clicks_entropy.rename(columns={'PROBA':'ENTROPY_CLICKS'}),how = 'left', on=['COURSE_PK1','USER_PK1'])


duration_entropy['USER_PK1']=duration_entropy['USER_PK1'].astype(int)
duration_entropy['COURSE_PK1']=duration_entropy['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(duration_entropy.rename(columns={'PROBA':'ENTROPY_SESSION_LENGTH'}),
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])
weeks_with_active_first_day['USER_PK1']=weeks_with_active_first_day['USER_PK1'].astype(int)
weeks_with_active_first_day['COURSE_PK1']=weeks_with_active_first_day['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(weeks_with_active_first_day.rename(columns={'PROPORTION_ACTIVE_WEEKS':
                                    'PROPORTION_WEEKS_FIRST_DAY_ACTIVE'})[['COURSE_PK1','USER_PK1',
                                    'PROPORTION_WEEKS_FIRST_DAY_ACTIVE']], how = 'left', on=['COURSE_PK1','USER_PK1'])
average_proportion_la_first_day['USER_PK1']=average_proportion_la_first_day['USER_PK1'].astype(int)
average_proportion_la_first_day['COURSE_PK1']=average_proportion_la_first_day['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(average_proportion_la_first_day.rename(columns={'PROPORTION':'PROPORTION_LA_FIRST_DAY_OF_WEEK'}),
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])

posts_per_day['USER_PK1']=posts_per_day['USER_PK1'].astype(int)
posts_per_day['COURSE_PK1']=posts_per_day['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(posts_per_day.rename(columns={'PROBA':'ENTROPY_FORUM_CONTRIBUTION_DAILY'}),
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])
posts_per_week['USER_PK1']=posts_per_week['USER_PK1'].astype(int)
posts_per_week['COURSE_PK1']=posts_per_week['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(posts_per_week.rename(columns={'PROBA':'ENTROPY_FORUM_CONTRIBUTION_WEEKLY'}),
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])

sessions_per_week_diversity['USER_PK1']=sessions_per_week_diversity['USER_PK1'].astype(int)
sessions_per_week_diversity['COURSE_PK1']=sessions_per_week_diversity['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(sessions_per_week_diversity.rename(columns={'DIVERSITY':
                                'DIVERSITY_OVERALL'})[['COURSE_PK1','USER_PK1','DIVERSITY_OVERALL']],
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])
sessions_per_week_diversity_exam['USER_PK1']=sessions_per_week_diversity_exam['USER_PK1'].astype(int)
sessions_per_week_diversity_exam['COURSE_PK1']=sessions_per_week_diversity_exam['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(sessions_per_week_diversity_exam[['COURSE_PK1','USER_PK1','DIVERSITY_EXAM_PERIOD']],
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])

sessions_total_in_top3_weeks['USER_PK1']=sessions_total_in_top3_weeks['USER_PK1'].astype(int)
sessions_total_in_top3_weeks['COURSE_PK1']=sessions_total_in_top3_weeks['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(sessions_total_in_top3_weeks.rename(columns={'LOYALTY':
                                'LOYALTY_OVERALL'})[['COURSE_PK1','USER_PK1','LOYALTY_OVERALL']],
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])
sessions_total_in_exam_weeks['USER_PK1']=sessions_total_in_exam_weeks['USER_PK1'].astype(int)
sessions_total_in_exam_weeks['COURSE_PK1']=sessions_total_in_exam_weeks['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(sessions_total_in_exam_weeks.rename(columns={'LOYALTY':
                                'LOYALTY_EXAM'})[['COURSE_PK1','USER_PK1','LOYALTY_EXAM']],
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])
regularity_df['USER_PK1']=regularity_df['USER_PK1'].astype(int)
regularity_df['COURSE_PK1']=regularity_df['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(regularity_df[['COURSE_PK1','USER_PK1','REGULARITY']],
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])
print(course_grades)

In [ ]:
n_weeks = len(set(course_data_non_zero['WEEK and YEAR']))

In [ ]:
course_grades['ZERO_SESSIONS_COUNT'] = course_grades['ZERO_SESSIONS_COUNT'].fillna(0)
course_grades['NON_ZERO_SESSION_COUNT'] = course_grades['NON_ZERO_SESSION_COUNT'].fillna(0) # OLA_1
course_grades['TOTAL_SESSION_DURATION'] = course_grades['TOTAL_SESSION_DURATION'].fillna(0) # OLA_2
course_grades['AVERAGE_SESSION_DURATION'] = course_grades['AVERAGE_SESSION_DURATION'].fillna(0) # OLA_3
course_grades['AVERAGE_ACTIONS_PER_SESSION'] = course_grades['AVERAGE_ACTIONS_PER_SESSION'].fillna(0) # OLA_4
course_grades['PROPORTION_ACTIVE_WEEKS'] = course_grades['PROPORTION_ACTIVE_WEEKS'].fillna(0) # OLA_8
course_grades['ACTIVE_DAYS_PROPORTION'] = course_grades['ACTIVE_DAYS_PROPORTION'].fillna(0) # OLA_5
course_grades['AVERAGE_ACTIVE_DAYS_PER_WEEK'] = course_grades['AVERAGE_ACTIVE_DAYS_PER_WEEK'].fillna(0) # OLA_6
course_grades['AVG_TIME_DIFF_BETWEEN_ACTIVE_DAYS'] = course_grades['AVG_TIME_DIFF_BETWEEN_ACTIVE_DAYS'].fillna(course_duration) # OLA_7
course_grades['ACTIVE_DAYS_PROPORTION_FORUM_CONTRIBUTION'] = course_grades['ACTIVE_DAYS_PROPORTION_FORUM_CONTRIBUTION'].fillna(0) # LALA_4
course_grades['PROPORTION_ACTIVE_WEEKS_FORUM_CONTRIBUTION'] = course_grades['PROPORTION_ACTIVE_WEEKS_FORUM_CONTRIBUTION'].fillna(0) # LALA_8
course_grades['PROPORTION_POSTS_READ'] = course_grades['PROPORTION_POSTS_READ'].fillna(0) # Forum consumption
course_grades['ENTROPY_CLICKS'] = course_grades['ENTROPY_CLICKS'].fillna(math.log2(median_number_sessions)) # ORS_1
course_grades['ENTROPY_SESSION_LENGTH'] = course_grades['ENTROPY_SESSION_LENGTH'].fillna(math.log2(median_number_sessions)) # ORS_2
course_grades['PROPORTION_WEEKS_FIRST_DAY_ACTIVE'] = course_grades['PROPORTION_WEEKS_FIRST_DAY_ACTIVE'].fillna(0) # ORS_3
course_grades['PROPORTION_LA_FIRST_DAY_OF_WEEK'] = course_grades['PROPORTION_LA_FIRST_DAY_OF_WEEK'].fillna(0) # ORS_4
course_grades['ENTROPY_FORUM_CONTRIBUTION_WEEKLY'] = course_grades['ENTROPY_FORUM_CONTRIBUTION_WEEKLY'].fillna(math.log2(n_weeks)) # LARS_8
course_grades['DIVERSITY_OVERALL'] = course_grades['DIVERSITY_OVERALL'].fillna(math.log2(n_weeks)/math.log2(n_weeks)) # diversity overall
course_grades['DIVERSITY_EXAM_PERIOD'] = course_grades['DIVERSITY_EXAM_PERIOD'].fillna(math.log2(len(exam_weeks))/math.log2(len(exam_weeks))) # diversity exam
course_grades['LOYALTY_OVERALL'] = course_grades['LOYALTY_OVERALL'].fillna(1) # loyalty
course_grades['LOYALTY_EXAM'] = course_grades['LOYALTY_EXAM'].fillna(1) # loyalty exam
course_grades['REGULARITY'] = course_grades['REGULARITY'].fillna(1) # regularity
course_grades['POSTS_CREATED'] = course_grades['POSTS_CREATED'].fillna(0) # forum contribution

print(course_grades)

In [ ]:
# Merging our features together with the given features
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
# Making the correct datatypes
course_grades['Score juni'] = course_grades['Score juni'].astype(int)
course_grades['USER_PK1'] = course_grades['USER_PK1'].astype(str)
# Merging it all together
merged_data1 = pd.merge(course_grades,workingdata_FI1 , on=["USER_PK1"], how='left')
merged_data1 = pd.merge(merged_data1, workingdata_FI2, on=["USER_PK1"] , how='left')
merged_data1 = pd.merge(merged_data1, workingdata_FI3, on=["USER_PK1"], how='left')
merged_data1 = pd.merge(merged_data1, workingdata_FI4, on=["USER_PK1"], how='left')
merged_data1 = pd.merge(merged_data1,workingdata_FI5, on=["USER_PK1"], how='left')
merged_data1 = pd.merge(merged_data1, workingdata_FT2, on=["USER_PK1"], how='left')
merged_data1 = pd.merge(merged_data1, workingdata_FT5, on=["USER_PK1"], how='left')
merged_data1 = pd.merge(merged_data1, work, on=["USER_PK1"], how='left')
merged_data1 = pd.merge(merged_data1, workingdata_FR1, on=["USER_PK1"], how='left')
merged_data1 = pd.merge(merged_data1, workingdata_FR2, on=["USER_PK1"], how='left')
merged_data1 = pd.merge(merged_data1, workingdata_FR3, on=["USER_PK1"], how='left')
merged_data1 = pd.merge(merged_data1, workingdata_FR4, on=["USER_PK1"], how='left')
# Table Check
print(merged_data1)
# Filling NaN
merged_data1['Score_bins'] = merged_data1['Score_bins'].astype(str).fillna(0)
merged_data1.fillna(0, inplace=True)
print(merged_data1)

# ------- TEST SET PREPARATION -------


### Preprocessing

In [ ]:
#_15_attempts_and_grades.xlsx
print(test_sets['test_df15'].info())
test_sets['test_df15']["ATTEMPT_START_DATE"] = pd.to_datetime(test_sets['test_df15']["ATTEMPT_START_DATE"],dayfirst=True)
test_sets['test_df15']["ATTEMPT_DATE"] = pd.to_datetime(test_sets['test_df15']["ATTEMPT_DATE"],dayfirst=True)
test_sets['test_df15']["COURSE_PK1"] = test_sets['test_df15']["COURSE_PK1"].astype(str)
test_sets['test_df15']["CONTENT_PK1"] = test_sets['test_df15']["CONTENT_PK1"].astype(str)
test_sets['test_df15']["USER_PK1"] = test_sets['test_df15']["USER_PK1"].astype(str)
test_sets['test_df15']["GRADEBOOK_COLUMN_PK1"] = test_sets['test_df15']["GRADEBOOK_COLUMN_PK1"].astype(str)
one_hot_encoded2 = pd.get_dummies(test_sets['test_df15']['ASSESSMENT_TYPE'], prefix='ASSESSMENT_TYPE_attemptsandgrades')
test_sets['test_df15'] = pd.concat([test_sets['test_df15'], one_hot_encoded2], axis=1)
df15 = test_sets['test_df15']
print(df15.info())
df15['CONTENT_PK1'].fillna('nan',inplace=True)
df15_filtered_FR4 = df15[df15['CONTENT_PK1'] != 'nan']
print(df15_filtered_FR4.info())

In [ ]:
#_16_posts_info.xlsx
print(test_sets['test_df16'].info())
test_sets['test_df16']["DTCREATED_postsinfo"] = pd.to_datetime(test_sets['test_df16']["DTCREATED"],dayfirst=True)
test_sets['test_df16']["DTMODIFIED_postsinfo"] = pd.to_datetime(test_sets['test_df16']["DTMODIFIED"],dayfirst=True)
test_sets['test_df16']['BINNED_MSG_TEXT_LENGTH'] = pd.cut(test_sets['test_df16']['MSG_TEXT_LENGTH'],bins=[-np.inf, 10, 50,100,200,400, np.inf],labels=["<10","10-50","50-100","100-200","200-400",">400"])
test_sets['test_df16']["CONTEXT_PK1"] = test_sets['test_df16']["CONTEXT_PK1"].astype(str)
test_sets['test_df16']["THREAD_PK1"] = test_sets['test_df16']["THREAD_PK1"].astype(str)
test_sets['test_df16']["REPLY_TO_POST_PK1"] = test_sets['test_df16']["REPLY_TO_POST_PK1"].astype(str)
test_sets['test_df16']["POST_PK1"] = test_sets['test_df16']["POST_PK1"].astype(str)
test_sets['test_df16']["USER_PK1"] = test_sets['test_df16']["USER_PK1"].astype(str)

In [ ]:
#_17_student_post_interaction.xlsx
print(test_sets['test_df17'].info())
test_sets['test_df17']["COURSE_PK1"] = test_sets['test_df17']["COURSE_PK1"].astype(str)
test_sets['test_df17']["USER_PK1"] = test_sets['test_df17']["USER_PK1"].astype(str)
test_sets['test_df17']["CONTEXT_PK1"] = test_sets['test_df17']["CONTEXT_PK1"].astype(str)
one_hot_encoded3 = pd.get_dummies(test_sets['test_df17']['ACTIVITYTYPE'], prefix='ACTIVITYTYPE')
test_sets['test_df17'] = pd.concat([test_sets['test_df17'], one_hot_encoded3], axis=1)
test_sets['test_df17']['binned_NUM_READ_POSTS'] = pd.cut(test_sets['test_df17']['NUM_READ_POSTS'],bins=[-np.inf, 5, 10,20,50, np.inf],labels=["<5","5-10","10-20","20-50",">50"])

In [ ]:
#_18_student_individual_posts_interaction.xlsx
print(test_sets['test_df18'].info())
test_sets['test_df18']["FIRST_VIEW"] = pd.to_datetime(test_sets['test_df18']["FIRST_VIEW"],dayfirst=True)
test_sets['test_df18']["LAST_VIEW"] = pd.to_datetime(test_sets['test_df18']["LAST_VIEW"],dayfirst=True)
test_sets['test_df18']["COURSE_PK1"] = test_sets['test_df18']["COURSE_PK1"].astype(str)
test_sets['test_df18']["CONTEXT_PK1"] = test_sets['test_df18']["CONTEXT_PK1"].astype(str)
test_sets['test_df18']["MESSAGE_PK1"] = test_sets['test_df18']["MESSAGE_PK1"].astype(str)
test_sets['test_df18']["USER_PK1"] = test_sets['test_df18']["USER_PK1"].astype(str)
one_hot_encoded4 = pd.get_dummies(test_sets['test_df18']['ACTIVITY_TYPE'], prefix='ACTIVITY_TYPE')
test_sets['test_df18'] = pd.concat([test_sets['test_df18'], one_hot_encoded4], axis=1)

In [ ]:
#_19_activity_accumulator_logs.csv
print(test_sets['test_df19'].info())
test_sets['test_df19']["TIMESTAMP_activity_accumulator_logs"] = pd.to_datetime(test_sets['test_df19']["TIMESTAMP"],dayfirst=True)
test_sets['test_df19']["COURSE_PK1"] = test_sets['test_df19']["COURSE_PK1"].astype(str)
test_sets['test_df19']["USER_PK1"] = test_sets['test_df19']["USER_PK1"].astype(str)
test_sets['test_df19']["CONTENT_PK1"] = test_sets['test_df19']["CONTENT_PK1"].astype(str)
test_sets['test_df19']["SESSION_ID"] = test_sets['test_df19']["SESSION_ID"].astype(str)

In [ ]:
#_21_toledo_user_activity_logs.csv
print(test_sets['test_df21'].info())
test_sets['test_df21']["TIMESTAMP_toledo_user_activity_logs"] = pd.to_datetime(test_sets['test_df21']["TIMESTAMP"], dayfirst=True)
test_sets['test_df21']["COURSE_PK1"] = test_sets['test_df21']["COURSE_PK1"].astype(str)
test_sets['test_df21']["USER_PK1"] = test_sets['test_df21']["USER_PK1"].astype(str)
test_sets['test_df21']["CONTENT_PK1"] = test_sets['test_df21']["CONTENT_PK1"].astype(str)
one_hot_encoded6 = pd.get_dummies(test_sets['test_df21']['EVENT'], prefix='EVENT')
test_sets['test_df21'] = pd.concat([test_sets['test_df21'], one_hot_encoded6], axis=1)

In [ ]:
# Preprocessing Y-variabele
workgrades = test_sets['test_df_grades'][['USER_PK1','OPO_ID','Score januari','Score juni']]
workgrades['Score juni'] = workgrades['Score juni'].replace({'': 0, '#': 0})
workgrades['Score juni'] = workgrades['Score juni'].replace([np.nan, np.inf, -np.inf], 0)
workgrades['Score juni'] = pd.to_numeric(workgrades['Score juni'])

bins = [0,7,9,15,20]
labels = ['fail (0-7)','deliberation (8-9)','pass (10-15)','very good (16-20)']
workgrades['Score_bins'] = pd.cut(workgrades['Score juni'],bins=bins,labels=labels,include_lowest=True)

print((workgrades['Score_bins'] == 'fail (0-7)').min())
print(workgrades.info())
print(workgrades.head())
workgrades['Score_bins'] = workgrades['Score_bins'].astype('category')
workgrades_encoded = pd.get_dummies(workgrades, columns=['Score_bins'])
print(workgrades_encoded)
workgrades= pd.merge(workgrades, workgrades_encoded, on=['USER_PK1','Score januari','Score juni','OPO_ID'])
workgrades['USER_PK1'] = workgrades['USER_PK1'].astype(str)
print(workgrades)

# Feature Engineering

## **Time Management and Regularity Features**

Paper 1: "Profiling students' self-regulation with learning analytics: a proof of concept", by Liz-Dominguez et al. (2022)

Paper 2: "Predictive power of regularity of pre-class activities in a flipped classroom" by Jovanovic et al. (2019)

### Feature N°1: PERCENTAGE_WEEKS_ACTIVITY_ABOVE_MEDIAN
"The relative amount of weeks in which the activity of the user was higher than the median of all students’ activities that week. With activity each event in file 19"

In [ ]:
time19 = test_sets['test_df19']['TIMESTAMP_activity_accumulator_logs']
test_sets['test_df19']['week_of_year'] = time19.dt.isocalendar().week
work19 = test_sets['test_df19'][['USER_PK1','week_of_year']]
weeklygrouped = work19.groupby(['USER_PK1','week_of_year']).size().reset_index(name='user_count')

Boxplots representing the user count for each "week_of_year" value representing the active students that week

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.boxplot(x='week_of_year',y='user_count',data=weeklygrouped)

A way to measure procrastiantion of students. Using median of medians of TRAIN set.

In [ ]:
# Median of medians of all weeks
if 'user_count_weekly_median' not in weeklygrouped.columns:
    weeklygrouped = weeklygrouped.merge(median_per_week_train, on='week_of_year', suffixes=('','_weekly_median'))

print(weeklygrouped.info())

# Was the activity of user x in week y higher than the median activity of all users that week?
weeklygrouped['above_median'] = weeklygrouped['user_count'].values > weeklygrouped['user_count_weekly_median'].values
print(weeklygrouped.head())

# How many weeks in total were user x's activity above the weekly median?
above_median_count = weeklygrouped.groupby('USER_PK1')['above_median'].sum().reset_index()
above_median_count.columns = ['USER_PK1','above_median_count']
print(above_median_count.head())

num_weeks = weeklygrouped['week_of_year'].nunique()
print(num_weeks)

above_median_count['percentageweeks_above_median'] = (above_median_count['above_median_count']/num_weeks)*100
print(above_median_count.head(20))
print(above_median_count.info())

# Merging the data with the students' results
test_sets['test_df_grades']["USER_PK1"] = test_sets['test_df_grades']["USER_PK1"].astype(str)
dfG = workgrades
dfG['USER_PK1'] = dfG['USER_PK1'].astype(str)

merge = pd.merge(above_median_count, dfG, on = 'USER_PK1', how = 'inner')
work = merge[['USER_PK1','above_median_count','percentageweeks_above_median','Score_bins']]

### Feature N°2: DAYS_UNTIL_FIRST_LOGIN
"Number of days it took each student to login on the course page for the first time. Taking the start of the semester as a

In [ ]:
# Weekly activity (number of events) by students
time19 = test_sets['test_df19']
print(time19.head())
grouped_data = time19.groupby('USER_PK1')

start = pd.to_datetime('2021-02-08')

In [ ]:
first_login = grouped_data['TIMESTAMP_activity_accumulator_logs'].min()
days_until_first_login = (first_login - start).dt.days
print(days_until_first_login)

workgrades["USER_PK1"] = workgrades["USER_PK1"].astype(str)
dfG = workgrades

# Merging the data with the students' results
merged = dfG.merge(days_until_first_login.reset_index(name='days_until_first_login'), on='USER_PK1', how='inner')
workingdata_FT2 = merged[['USER_PK1','days_until_first_login']]
print(workingdata_FT2.head())

### Feature N°3: ENTROPY_WEEKLY_SESSION_COUNTS
"Entropy of weekly session counts per student. Once again, the sessions are here the events as in file 19, with no operations performed on these"

In [ ]:
weekly_session_counts_per_student = time19.groupby(['USER_PK1', 'week_of_year']).size().unstack(fill_value=0)

In [ ]:
import numpy as np
from scipy.stats import entropy

# Calculating probability distribution for each student
prob_distribution_per_student = weekly_session_counts_per_student.apply(lambda x: x / x.sum(), axis=1)

# Calculating entropy for each student in the first semester
entropy_per_student = prob_distribution_per_student.apply(lambda x: entropy(x, base=2) if x.sum() > 0 else 0, axis=1)

print(f"Entropy of weekly session counts per student for the first semester:")
print(entropy_per_student)

# Merging the data with the students' results
workgrades["USER_PK1"] = workgrades["USER_PK1"].astype(str)
dfG = workgrades
merged = dfG.merge(entropy_per_student.reset_index(name='entropy_per_student'), on='USER_PK1', how='inner')
workingdata_FT5 = merged[['USER_PK1','entropy_per_student']]
print(workingdata_FT5.head())

## Regularity features

Paper 1: "Differences by course discipline on student behaviorn persistence, and achievement in online courses of undergraduate general education" by Finnegan et al.

Paper 2: "Predictive power of regularity of pre-class activities in a flipped classroom" by Jovanovic et al.

Paper 3: "How learning analytics can early predict under-achieving students in a blended medical education course" by Saqr et al.

Paper 4: "Students matter the most in learning analytics: The effects of internal and instructional conditions in predicting academic success" by Jovanovic et al.

### Feature N°1: FREQ_CHANGE_ENGAGEMENT_PATTERN
"Frequency of change in a student’s engagement pattern over the days of the week"

In [ ]:
# Limit session duration to max. 8 hours
df19 = test_sets['test_df19']
print(df19.info())

session_duration = df19.groupby("SESSION_ID")["TIMESTAMP_activity_accumulator_logs"].agg(['min','max'])
session_duration['duration'] = session_duration['max'] - session_duration['min']
# Converting column to hours
session_duration['duration_hours'] = session_duration['duration'] / pd.Timedelta(hours=1)
# Limiting durations longer than 8 hours to 8 hours
session_duration['duration_hours'] = session_duration['duration_hours'].clip(upper=8)
# Converting duration back to normal format
session_duration['duration'] = pd.to_timedelta(session_duration['duration_hours'], unit='h')
print(session_duration.head())

In [ ]:
# Creation of DataFrame with daily activity relative to their weekly total for each student

# Extracting week number
df19['week_number'] = df19['TIMESTAMP_activity_accumulator_logs'].dt.isocalendar().week

# Extracting day of the week
df19['day_of_week'] = df19['TIMESTAMP_activity_accumulator_logs'].dt.dayofweek + 1

# Specifying the Semester
work19_FR1 = df19[(df19['week_number']>=6) & (df19['week_number']<=25)]
all_weeks = list(range(6,26))

# Grouping the data based on the student, the week number and the day of the week
grouped_data_FR1 = work19_FR1.groupby(['USER_PK1', 'week_number', 'day_of_week'])

# Calculating total sessions per day per week per student
total_sessions_per_day_per_week_per_student = grouped_data_FR1['SESSION_ID'].nunique().unstack(fill_value=0)
print(total_sessions_per_day_per_week_per_student.head(10))

# Calculating total sessions per week per student
total_sessions_per_week_per_student = total_sessions_per_day_per_week_per_student.sum(axis=1)

# Reindexing to ensure all combinations of students and weeks are included, filling missing values with zeros
all_students = work19_FR1['USER_PK1'].unique()
new_index = pd.MultiIndex.from_product([all_students, all_weeks], names=['USER_PK1', 'week_number'])
total_sessions_per_day_per_week_per_student = total_sessions_per_day_per_week_per_student.reindex(new_index, fill_value=0)

# Replacing NaN values in weeks with no activity with 0
total_sessions_per_day_per_week_per_student.fillna(0, inplace=True)

# Recalculating total sessions per week per student after reindexing
total_sessions_per_week_per_student = total_sessions_per_day_per_week_per_student.sum(axis=1)
print(total_sessions_per_week_per_student.head())

# Calculating relative weights per week per student
relative_weights_per_week_per_student = total_sessions_per_day_per_week_per_student.div(total_sessions_per_week_per_student, axis=0)

# Replacing NaN values in weeks with no activity with 0
relative_weights_per_week_per_student.fillna(0,inplace=True)

print(relative_weights_per_week_per_student.head())

# Calculating the mean squared difference between the vectors of two consecutive weeks
# Calculating the difference between consecutive weeks
diff_between_weeks = relative_weights_per_week_per_student.diff(axis=0)

# Squaring each difference
squared_diff = diff_between_weeks ** 2

# Calculating the mean squared difference for each student
mean_squared_diff_per_student = squared_diff.mean(axis=1)

# Calculating the aggregate number over all weeks for each student
aggregate_mean_squared_diff_per_student = mean_squared_diff_per_student.groupby('USER_PK1').sum()

print("Mean squared difference per student between consecutive weeks:")
print(mean_squared_diff_per_student.head())

print("\nAggregate mean squared difference over all weeks for each student:")
print(aggregate_mean_squared_diff_per_student)

# Merging the data with the students' results
workgrades["USER_PK1"] = workgrades["USER_PK1"].astype(str)
dfG = workgrades
merged_FR1 = dfG.merge(aggregate_mean_squared_diff_per_student.reset_index(name='aggregate_mean_squared_diff_per_student'), on='USER_PK1', how='inner')
print(merged_FR1.head())
workingdata_FR1 = merged_FR1[['USER_PK1', 'aggregate_mean_squared_diff_per_student']]
print(workingdata_FR1.head())

### Feature N°2: WEEKLY_LOGIN_ENGAGEMENT
"Measuring engagement by looking at login behavior. A student was considered engaged in a certain week when having logged in 3 days or more in that week"

In [ ]:
df19 = test_sets['test_df19']

# 1. Creating a new column to register the active days per week
df19['week_number'] = df19['TIMESTAMP_activity_accumulator_logs'].dt.isocalendar().week
df19['day_of_week'] = df19['TIMESTAMP_activity_accumulator_logs'].dt.dayofweek + 1

# 2. Grouping the data based on the student and the week number
grouped_data_FR2 = df19.groupby(['USER_PK1', 'week_number'])

# 3. Calculating the total active days per week for each student
active_days_per_week = grouped_data_FR2['day_of_week'].nunique()
print(active_days_per_week)

# 4. Assigning a score of 1 or 0 based on whether the total active days per week is >= 3
weekly_scores = (active_days_per_week >= 3).astype(int)
print(weekly_scores)

# 5. Summing the weekly scores to calculate the total score for each student
total_scores_per_student_FR2 = weekly_scores.groupby('USER_PK1').sum().reset_index(name='total_score_FR2')

print(total_scores_per_student_FR2)

# Merging the data with the students' results
workgrades["USER_PK1"] = workgrades["USER_PK1"].astype(str)
dfG = workgrades
merged_FR2 = dfG.merge(total_scores_per_student_FR2.reset_index(), on='USER_PK1', how='inner')
print(merged_FR2.head())
workingdata_FR2 = merged_FR2[['USER_PK1', 'total_score_FR2']]
print(workingdata_FR2.head())

### Feature N°3: FORUM_ENGAGEMENT
"Measuring engagement by looking at the forum posts views. A score of one was assigned when a student views the course materials more than a Z-score of mean course views (using -1.96 and +1.96 for a two-sided 95% confidence interval)"

In [ ]:
df18 = test_sets['test_df18']

num_content_items = df18['MESSAGE_PK1'].nunique()
print(num_content_items)

message_avg_views = df18.groupby('MESSAGE_PK1')['NUM_VIEWS'].mean()
df18 = df18.merge(message_avg_views, on='MESSAGE_PK1', suffixes=('','_avg'))
df18['Z_score'] = (df18['NUM_VIEWS'] - df18['NUM_VIEWS_avg']) / df18['NUM_VIEWS'].std()

Z_score_threshold = 1.96
df18['exceptional_view'] = np.where(df18['Z_score'] > Z_score_threshold, 1, 0)

student_scores_FR3 = df18.groupby('USER_PK1')['exceptional_view'].sum().reset_index()
student_scores_FR3['exceptional_view_percentage'] = round((student_scores_FR3['exceptional_view'] / num_content_items) * 100, 2)
print(student_scores_FR3.head(30))

# Merging the data with the students' results
workgrades["USER_PK1"] = workgrades["USER_PK1"].astype(str)
dfG = workgrades
merged_FR3 = dfG.merge(student_scores_FR3.reset_index(), on='USER_PK1', how='inner')
print(merged_FR3.head())
workingdata_FR3 = merged_FR3[['USER_PK1', 'exceptional_view']]
print(workingdata_FR3.head())

### Feature N°4: ASSESSMENT_ENGAGEMENT
"Measuring engagement by looking at assessment attempts"

In [ ]:
# Creation of column where each row represents an attempt of a specific USER_PK1 & CONTENT_PK1 combination
df15_filtered_FR4['attempts'] = 1
attempts_counts_FR4 = df15_filtered_FR4.groupby(['USER_PK1','CONTENT_PK1'])['attempts'].size().reset_index(name=('attempts'))
print(attempts_counts_FR4[attempts_counts_FR4['attempts']>1])
print(attempts_counts_FR4.head())
total_attempts_per_student_FR4 = attempts_counts_FR4.groupby('USER_PK1')['attempts'].sum().reset_index(name='total_score')
print(total_attempts_per_student_FR4)
# Look at each unique value of CONTENT_PK1 and check whether this appears in the students' attempts
content_items = df15['CONTENT_PK1'].unique()
print(len(content_items)) # Number of content items in total

user_scores_FR4 = {}

for content in content_items:
    content_data = df15[df15['CONTENT_PK1'] == content]
    content_data = content_data.merge(attempts_counts_FR4, on=['USER_PK1', 'CONTENT_PK1'], how='left')
    processed_users = set() # Keep track of processed users such that a user-content item combination is not counted twice

    for index, row in content_data.iterrows():
        user = row['USER_PK1']

        if user in processed_users:
            continue # Skipping the following loop if user has already been processed for this content item

        # Does the user already exist in the dictionary?!
        if user in user_scores_FR4:
            user_scores_FR4[user] += 1 if row['attempts'] > 0 else 0
        else:
            user_scores_FR4[user] = 1 if row['attempts'] > 0 else 0

        processed_users.add(user) # Adding the user to the set of processed users

total_scores_FR4 = pd.DataFrame(list(user_scores_FR4.items()), columns=['USER_PK1','total_score'])

print(total_scores_FR4)

# Merging the data with the students' results
workgrades["USER_PK1"] = workgrades["USER_PK1"].astype(str)
dfG = workgrades
merged_FR4 = dfG.merge(total_scores_FR4.reset_index(), on='USER_PK1', how='inner')
print(merged_FR4.head())
workingdata_FR4 = merged_FR4[['USER_PK1', 'total_score']]
print(workingdata_FR4.head())

## **INTERACTION FEATURES**

Paper: "Learning at distance: Effects of interaction traces on academic achievement" by Joksimovic et al. (2015)

In [ ]:
df17 = test_sets['test_df17']
grouped_data = df17.groupby('USER_PK1')

### Feature N°1: TOTAL_NUM_POSTS_READ
"Number of forum posts read"

In [ ]:
sum_read_posts = grouped_data['NUM_READ_POSTS'].sum()
print(sum_read_posts)

# Merging the data with the students' results
workgrades["USER_PK1"] = workgrades["USER_PK1"].astype(str)
dfG = workgrades
merged = dfG.merge(sum_read_posts.reset_index(name='sum_read_posts'), on='USER_PK1', how='inner')
workingdata_FI1 = merged[['USER_PK1','sum_read_posts']]
print(workingdata_FI1.head())

### Feature N°2: TOTAL_NUM_COMMENTS
 "Number of comments on forum posts"

In [ ]:
sum_comments = grouped_data['NUM_REPLIES_COMMENTS'].sum()
print(sum_comments)

# Merging the data with the students' results
workgrades["USER_PK1"] = workgrades["USER_PK1"].astype(str)
dfG = workgrades
merged = dfG.merge(sum_comments.reset_index(name='sum_comments'), on='USER_PK1', how='inner')
workingdata_FI2 = merged[['USER_PK1','sum_comments']]
print(workingdata_FI2.head())

In [ ]:
df18 = test_sets['test_df18']

grouped_data = df18.groupby('USER_PK1')

### Feature N°3: TOTAL_NUM_VIEWS
"Total number of views on posts"

In [ ]:
sum_views = grouped_data['NUM_VIEWS'].sum()
print(sum_views)

# Merging the data with the students' results
workgrades["USER_PK1"] = workgrades["USER_PK1"].astype(str)
dfG = workgrades
merged = dfG.merge(sum_views.reset_index(name='sum_views'), on='USER_PK1', how='inner')
workingdata_FI3 = merged[['USER_PK1','sum_views']]
print(workingdata_FI3.head())

### Feature N°4: TOTAL_POSTS_INDICATED_READ
"Number of posts indicated as read"

In [ ]:
sum_ind_read = grouped_data['READ_STATE_IND'].sum()
print(grouped_data.head())

workgrades["USER_PK1"] = workgrades["USER_PK1"].astype(str)
dfG = workgrades

# Merging the data with the students' results
merged = dfG.merge(sum_ind_read.reset_index(name='sum_ind_read'), on='USER_PK1', how='inner')
workingdata_FI4 = merged[['USER_PK1','sum_ind_read']]
print(workingdata_FI4.head())

### Feature N°5: MEAN_TEXT_LENGTH_OF_POSTS
"Mean of students’ post length"

In [ ]:
df16 = test_sets['test_df16']
grouped_data = df16.groupby('USER_PK1')

In [ ]:
mean_text_length = grouped_data['MSG_TEXT_LENGTH'].mean()
print(mean_text_length)

workgrades["USER_PK1"] = workgrades["USER_PK1"].astype(str)
dfG = workgrades

# Merging the data with the students' results
merged = dfG.merge(mean_text_length.reset_index(name='mean_text_length'), on='USER_PK1', how='inner')
workingdata_FI5 = merged[['USER_PK1','mean_text_length']]
print(workingdata_FI5.head())

## Second set of features

Paper: "Discovering Unusual Study Patterns Using Anomaly Detection and XAI" by Tiukhova et al. (2024)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import datetime
import datetime
import warnings
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
pd.set_option('display.max_columns', None)

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
from scipy.stats import entropy
pd.set_option('display.max_colwidth', None)

import math

In [ ]:
data_19 = test_sets['test_df19'].copy()

In [ ]:
data_19 = data_19.assign(TIMESTAMP_NEW=pd.to_datetime(data_19['TIMESTAMP'],dayfirst=True))
data_19['WEEK'] = data_19.apply(lambda x: x['TIMESTAMP_NEW'].isocalendar()[1], axis = 1)
data_19['YEAR'] =  data_19.apply(lambda x: x['TIMESTAMP_NEW'].isocalendar()[0], axis = 1)
data_19['WEEK'] = data_19['WEEK'].astype(str)
data_19['YEAR'] = data_19['YEAR'].astype(str)
data_19['WEEK and YEAR'] = 'Week ' + data_19['WEEK'] + ' of ' + data_19['YEAR']
data_19 = data_19.drop(columns = ['Unnamed: 0'], errors = 'ignore')

data_19['CONTENT_PK1'] = data_19['CONTENT_PK1'].fillna(0)
data_19['CONTENT_PK1'] = data_19['CONTENT_PK1'].astype(int)

In [ ]:
test_sets['test_df7']['CONTENT_PK1'] = test_sets['test_df7']['CONTENT_PK1'].astype(str)
test_sets['test_df7']['COURSE_PK1'] = test_sets['test_df7']['COURSE_PK1'].astype(str)
data_19['CONTENT_PK1'] = data_19['CONTENT_PK1'].astype(str)
data_19['COURSE_PK1'] = data_19['COURSE_PK1'].astype(str)

In [ ]:
# we merge activity data with the course content information to get more details on the activity
data_19 = data_19.merge(test_sets['test_df7'][['COURSE_PK1', 'CONTENT_PK1',
         'CONTENT_TYPE', 'TITLE', 'PATH']], on = ['COURSE_PK1', 'CONTENT_PK1'], how = 'left')
data_19['CONTENT_TYPE'] = data_19.apply(lambda x: x['DATA'] if pd.isna(x['CONTENT_TYPE']) else x['CONTENT_TYPE'], axis = 1)

In [ ]:
def new_session_id_func(session_ids, df):
    sessions= []
    for session_id in tqdm(session_ids):
        session = df[df['SESSION_ID'] == session_id]
        session = session.reset_index()
        indices = session[session['TIME_DIFF_SESSION']>= 7200].index
        n = len(indices)-1
        for i in sorted(indices,reverse=True):
            session.loc[:i-1, 'new_session_id'] = f'session_{session_id}_{n}'
            n = n - 1

        session['new_session_id'] = session['new_session_id'].fillna(f'session_{session_id}_{len(indices)}')
        sessions.append(session)
    return pd.concat(sessions)

In [ ]:
def change_sessions(data):
    data = data.sort_values(by = ['SESSION_ID', 'TIMESTAMP_NEW']) # first sort data within a session
    data['TIME_DIFF_SESSION'] = data.groupby(by = ['SESSION_ID'])['TIMESTAMP_NEW'].diff().dt.seconds.fillna(0) #calculate the time difference between learning activities within a session
    sessions_to_divide = set(data[data['TIME_DIFF_SESSION'] > 7200]['SESSION_ID']) #get the sessions where the difference between learning activities is larger than 2h
    imputed_sessions = new_session_id_func(sessions_to_divide, data[data['SESSION_ID'].isin(sessions_to_divide)]) #apply a function that will create a new session id - subsession
    imputed_sessions = imputed_sessions.drop(columns = 'index')
    imputed_sessions = imputed_sessions.sort_values(by = ['SESSION_ID', 'TIMESTAMP_NEW'])
    imputed_sessions = imputed_sessions.reset_index()
    imputed_sessions = imputed_sessions.drop(columns = 'index')

    left_session = data[~data['SESSION_ID'].isin(sessions_to_divide)] #get the rest of sessions that does not need to be separated
    left_session['new_session_id'] = left_session['SESSION_ID'].copy()
    new_data = pd.concat([imputed_sessions, left_session])
    return new_data

In [ ]:
data_19_new = pd.DataFrame()
for course in set(data_19['COURSE_PK1']):
    course_data = data_19[data_19['COURSE_PK1'] == course]
    course_data_new = change_sessions(course_data)
    data_19_new = pd.concat([data_19_new, course_data_new])

    print(course_data)

# each row in the table_19 represents an event (a unique combination of timestamp and content PK)
data_19_new['EVENTS'] = 1

print(data_19_new)

In [ ]:
# we take only those posts that we have interactions for.
data_16 = test_sets['test_df16'][test_sets['test_df16']['CONTEXT_PK1'].isin(list(set(test_sets['test_df17']['CONTEXT_PK1'])))]
data_16 = data_16.assign(DTCREATED_NEW=pd.to_datetime(data_16['DTCREATED'],dayfirst=True))
data_16['WEEK_CREATED'] = data_16.apply(lambda x: x['DTCREATED_NEW'].isocalendar()[1], axis = 1)
data_16['YEAR'] = data_16.apply(lambda x: x['DTCREATED_NEW'].isocalendar()[0], axis = 1)
data_16['WEEK_CREATED'] = data_16['WEEK_CREATED'].astype(str)
data_16['YEAR'] = data_16['YEAR'].astype(str)
data_16['WEEK and YEAR created'] = 'Week ' + data_16['WEEK_CREATED'] + ' of ' + data_16['YEAR']

In [ ]:
data_17 = test_sets['test_df17'].copy()

workgrades["USER_PK1"] = workgrades["USER_PK1"].astype(str)
grades = workgrades

## FEATURE ENGINEERING

In [ ]:
# Semester 2
course_start, course_finish, exam_weeks, semester_weeks  = second_semester_start, end_sem2,('Week 21 of 2021','Week 22 of 2021','Week 23 of 2021','Week 24 of 2021','Week 25 of 2021'), ('Week 6 of 2021','Week 7 of 2021','Week 8 of 2021','Week 9 of 2021','Week 10 of 2021','Week 11 of 2021','Week 12 of 2021','Week 13 of 2021','Week 14 of 2021','Week 15 of 2021','Week 16 of 2021','Week 17 of 2021','Week 18 of 2021','Week 19 of 2021','Week 20 of 2021')

print(course_start)
print(exam_weeks)

course_duration = (course_finish - course_start).days
course_duration_weeks = course_duration/7

print(course_duration_weeks)

In [ ]:
course_data = data_19_new
# filter out the data outside the course timespan
course_data = course_data[(course_data['TIMESTAMP_NEW'] > course_start) &(course_data['TIMESTAMP_NEW'] < course_finish)]
course_data['DATE'] = course_data['TIMESTAMP_NEW'].dt.date
course_data['DATE'] = pd.to_datetime(course_data['DATE'])
course_data['PATH'] = course_data['PATH'].fillna('not specified')

print(course_data)

course_data_posts = data_16
# we only want to take into account the posts of the current academic year
course_data_posts = course_data_posts[(course_data_posts['DTCREATED_NEW']>=course_start) & (course_data_posts['DTCREATED_NEW'] <= course_finish)]
course_data_posts['DATE_CREATED'] = course_data_posts['DTCREATED_NEW'].dt.date
print(course_data_posts)

course_data_posts_consume = data_17

## Overall Level of Activity

### Feature N°1: ZERO_SESSION_COUNT
"Number of sessions with duration of zero"

In [ ]:
# then we need to calculate each session's duration. To do that, we find the max and min timestamp value per session and substract latter from the former
# we do it to filter out very long/short sessions
session_duration = pd.pivot_table(course_data, values='TIMESTAMP_NEW', index=['COURSE_PK1','USER_PK1', 'new_session_id'],aggfunc='max').fillna(0) - pd.pivot_table(course_data, values='TIMESTAMP_NEW', index=['COURSE_PK1','USER_PK1', 'new_session_id'],aggfunc='min').fillna(0)
session_duration = session_duration.reset_index()

session_duration['SECONDS'] = session_duration['TIMESTAMP_NEW'] / np.timedelta64(1, 's')
session_duration['MINUTES'] = session_duration['TIMESTAMP_NEW'] / np.timedelta64(1, 'm')
session_duration['HOURS'] = session_duration['TIMESTAMP_NEW'] / np.timedelta64(1, 'h')

print(session_duration.info())

# we calculate the number of sessions with no duration - could be quick access to the announcements
session_count_zero_duration = pd.pivot_table(session_duration[session_duration['SECONDS'] == 0], values='new_session_id', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'new_session_id':'SESSION_COUNT'})

print(session_count_zero_duration.head())

plt.hist(session_count_zero_duration['SESSION_COUNT'], bins= 20)
plt.title('TOTAL ZERO LENGTH SESSION COUNT')

In [ ]:
# We exclude sessions of length 0
course_data_non_zero = course_data.drop(course_data[course_data['new_session_id'].isin(set(session_duration[session_duration['SECONDS'] == 0]['new_session_id']))].index).copy()

### Feature N°2: NON_ZERO_SESSION_COUNT
"Number of sessions, excluding the ones with duration of zero"

In [ ]:
# We cut sessions with a duration of more than 8 hours to a duration of 8 hours
session_duration['HOURS'] = np.where(session_duration['HOURS'] > 8, 8, session_duration['HOURS'])

# We exclude the sessions of duration of 0 
session_duration = session_duration.drop(session_duration[session_duration['SECONDS']  == 0].index) # including sessions with 0 duration can spoil the average
print(session_duration.count())

session_duration_plt = plt.hist(session_duration['MINUTES'])

In [ ]:
# getting session count - non-zero sessions
session_count = pd.pivot_table(course_data_non_zero, values='new_session_id', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'new_session_id':'SESSION_COUNT'})

print(session_count.head())

In [ ]:
median_number_sessions = int(session_count['SESSION_COUNT'].median())

print(median_number_sessions)

In [ ]:
plt.hist(session_count['SESSION_COUNT'], bins= 20)
plt.title('TOTAL SESSION COUNT')

###  Feature N°3: AVERAGE_ACTIONS_PER_SESSION
"Average number of  clicks / learning actions each session"

In [ ]:
# getting the number of events per timestamp for future filtering
clicks_per_timestamp = pd.pivot_table(course_data_non_zero, values='EVENTS', index=['COURSE_PK1','USER_PK1', 'new_session_id','TIMESTAMP_NEW'],
                     aggfunc='sum').fillna(0).reset_index()

clicks_per_timestamp.sort_values(by = 'EVENTS', ascending = False)

In [ ]:
# replace the rows with #events > 1 by 1
# as Toledo logs all the subfolders of an opened parent folder as a separate event, we replace this multiple events by just 1 event of openening a folder
mask = clicks_per_timestamp['EVENTS'] > 1
column_name = 'EVENTS'
clicks_per_timestamp.loc[mask, column_name] = 1
clicks_per_timestamp.sort_values(by = 'EVENTS', ascending = False)

In [ ]:
# getting the median number of events (learning actions) per sessions of a user
clicks_per_session = pd.pivot_table(clicks_per_timestamp, values='EVENTS', index=['COURSE_PK1','USER_PK1', 'new_session_id'],
                     aggfunc='sum').fillna(0).reset_index()
clicks_per_user = pd.pivot_table(clicks_per_session, values='EVENTS', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='median').fillna(0).reset_index().rename(columns = {'EVENTS':'CLICKS_PER_SESSION_AVG'})

print(clicks_per_session.head())
print(clicks_per_user.head()) 

plt.hist(clicks_per_user['CLICKS_PER_SESSION_AVG'], bins= 20)
plt.title('MEDIAN #LEARNING ACTIONS PER SESSION')

### Feature N°4: TOTAL_SESSION_DURATION
"The total duration of all sessions of each student (in seconds)"

In [ ]:
# getting total session length per user (in seconds)
session_duration_user = pd.pivot_table(session_duration, values='SECONDS', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='sum').fillna(0).reset_index().rename(columns = {'SECONDS':'SESSION_DURATION'})

plt.hist(session_duration_user['SESSION_DURATION'], bins= 20)
plt.title('TOTAL SESSION LENGTH')

### Feature N°5: AVERAGE_SESSION_DURATION
"The median session length of each user"

In [ ]:
# getting average (median) session length per user
session_duration_user_avg = pd.pivot_table(session_duration, values='SECONDS', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='median').fillna(0).reset_index().rename(columns = {'SECONDS':'SESSION_DURATION_AVG'})

plt.hist(session_duration_user_avg['SESSION_DURATION_AVG'], bins= 20)
plt.title('MEDIAN SESSION LENGTH - SECONDS')

### Feature N°6: PROPORTION_ACTIVE_WEEKS
"The proportion of weeks a user has had an active week - with this being defined as follows; a week is active when the number of active days is higher than the average"

In [ ]:
active_days = pd.pivot_table(course_data, values='DATE', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'DATE':'ACTIVE_DAYS'})

# getting the number of active days per week per user
active_days_per_week = pd.pivot_table(course_data, values='DATE', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'DATE':'ACTIVE_DAYS'})

print(active_days_per_week.info())

In [ ]:
# getting the average number of active days per week for this course
active_days_avg_course = pd.pivot_table(active_days_per_week, values='ACTIVE_DAYS', index=['COURSE_PK1'],
                     aggfunc='mean')
active_days_avg_course = active_days_avg_course.to_dict()

print(active_days_avg_course)

# leaving only those weeks with #active days higher than average #active days in a week per course
active_days_per_week = active_days_per_week[active_days_per_week.apply(lambda x: x['ACTIVE_DAYS'] >= active_days_avg_course_train['ACTIVE_DAYS'][x['COURSE_PK1']], axis = 1)]

print(active_days_per_week.info())

In [ ]:
# getting a proportion of active weeks
num_active_weeks = pd.pivot_table(active_days_per_week, values='WEEK and YEAR', index=['COURSE_PK1', 'USER_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'WEEK and YEAR':'ACTIVE_WEEKS'})
num_active_weeks['PROPORTION_ACTIVE_WEEKS'] = num_active_weeks['ACTIVE_WEEKS']/course_duration_weeks

print(num_active_weeks.head())

plt.hist(num_active_weeks['PROPORTION_ACTIVE_WEEKS'], bins= 10)
plt.title('PROPORTION OF ACTIVE WEEKS')

###  Feature N°7: ACTIVE_DAYS_PROPORTION
"By considering the course duration, a calculation is made on the amount of active days of each user"

In [ ]:
# getting a proportion of active days
active_days['ACTIVE_DAYS_PROPORTION'] = active_days['ACTIVE_DAYS']/course_duration

print(active_days.head())

plt.hist(active_days['ACTIVE_DAYS_PROPORTION'], bins= 20)
plt.title('PROPORTION OF ACTIVE DAYS')

### Feature N°8: AVERAGE_ACTIVE_DAYS_PER_WEEK
"The median number of active days per week for each user"

In [ ]:
# getting a median number of active days per week
active_days_per_week = pd.pivot_table(course_data, values='DATE', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR'],
                     aggfunc=pd.Series.nunique).reset_index()
active_days_per_week_per_user = pd.pivot_table(active_days_per_week, values='DATE', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='median').reset_index().rename(columns = {'DATE':'AVERAGE_ACTIVE_DAYS_PER_WEEK'})

print(active_days_per_week.head())
print(active_days_per_week_per_user.head())

plt.hist(active_days_per_week_per_user['AVERAGE_ACTIVE_DAYS_PER_WEEK'], bins= 10)
plt.title('MEDIAN OF #ACTIVE DAYS PER WEEK')

### Feature N°9: AVG_TIME_DIFF_BETWEEN_ACTIVE_DAYS
"For each user, the median time distance between two consecutive active days"

In [ ]:
course_data = course_data.sort_values(by = ['COURSE_PK1','USER_PK1', 'TIMESTAMP_NEW'])

max_day_session = pd.pivot_table(course_data, values='DATE', index=['COURSE_PK1','USER_PK1', 'new_session_id'],
                     aggfunc='max').reset_index().sort_values(by = ['COURSE_PK1','USER_PK1','DATE'])

max_day_session['DIFF'] = max_day_session['DATE'].diff().dt.days
max_day_session['DIFF'] = max_day_session['DIFF'].clip(lower=0)
max_day_session['DIFF'] = max_day_session['DIFF'].fillna(0)

print(max_day_session.head())

# We are considering only the date and drop the dubplicates so that we do not affect the difference between days when there are more than 1 active session
max_day_session = max_day_session.drop_duplicates(subset = ['COURSE_PK1','USER_PK1','DATE'], keep = 'first')

# Getting median time distance between two consecutive active days
avg_time_distance_user = pd.pivot_table(max_day_session, values='DIFF', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='median').reset_index().rename(columns = {'DIFF':'AVG_DIFF'})

print(avg_time_distance_user.head())

plt.hist(avg_time_distance_user['AVG_DIFF'], bins= 20)
plt.title('MEDIAN TIME DISTANCE BETWEEN CONSECUTIVE ACTIVE DAYS')

## Learning Action Specific Level of Activity

### Feature N°1: ACTIVE_DAYS_PROPORTION_FORUM_CONTRIBUTION
"Proportion of active days for forum contribution"

In [ ]:
# getting a proportion of active days for the posts written
active_days_posts_written = pd.pivot_table(course_data_posts, values='DATE_CREATED', index=['COURSE_PK1','USER_PK1'],aggfunc=pd.Series.nunique).reset_index().rename(columns = {'DATE_CREATED':'ACTIVE_DAYS'})
active_days_posts_written['ACTIVE_DAYS_PROPORTION'] = active_days_posts_written['ACTIVE_DAYS']/course_duration

plt.hist(active_days_posts_written['ACTIVE_DAYS_PROPORTION'], bins= 10)
plt.title('PROPORTION OF ACTIVE DAYS FOR FORUM CONTRIBUTION')

# getting the number of active days with posts written per week per user
active_days_per_week_posts_written = pd.pivot_table(course_data_posts, values='DATE_CREATED', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR created'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'DATE_CREATED':'ACTIVE_DAYS'})

In [ ]:
# getting the average number of active days per week per course
active_days_avg_posts_written = pd.pivot_table(active_days_per_week_posts_written, values='ACTIVE_DAYS', index=['COURSE_PK1'],
                     aggfunc='mean')
active_days_avg_posts_written = active_days_avg_posts_written.to_dict()

In [ ]:
# leaving only those weeks with #active days higher than average #active days in a week per course
active_days_per_week_posts_written = active_days_per_week_posts_written[active_days_per_week_posts_written.apply(lambda x: x['ACTIVE_DAYS'] >= active_days_avg_posts_written_train['ACTIVE_DAYS'][x['COURSE_PK1']], axis = 1)]

In [ ]:
# getting a proportion of active weeks for the writing posts activity type
num_active_weeks_posts_written = pd.pivot_table(active_days_per_week_posts_written, values='WEEK and YEAR created', index=['COURSE_PK1', 'USER_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'WEEK and YEAR created':'ACTIVE_WEEKS'})
num_active_weeks_posts_written['PROPORTION_ACTIVE_WEEKS'] = num_active_weeks_posts_written['ACTIVE_WEEKS']/course_duration_weeks

plt.hist(num_active_weeks_posts_written['PROPORTION_ACTIVE_WEEKS'], bins= 5)
plt.title('PROPORTION OF ACTIVE WEEKS FOR FORUM CONTRIBUTION')

###  Feature N°2: PROPORTION_POSTS_READ
"The proportion of posts a student has read"

In [ ]:
total_number_posts = pd.pivot_table(course_data_posts, values='POST_PK1', index=['COURSE_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'POST_PK1':'TOTAL'})

print(total_number_posts)

pd.pivot_table(course_data_posts, values='POST_PK1', index=['COURSE_PK1', 'CONTEXT_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'POST_PK1':'TOTAL'})

pd.pivot_table(course_data_posts_consume, values='NUM_READ_POSTS', index=['COURSE_PK1', 'CONTEXT_PK1'],
                     aggfunc='max').reset_index()

pd.pivot_table(course_data_posts, values='POST_PK1', index=['COURSE_PK1', 'CONTEXT_PK1'],
                     aggfunc=pd.Series.nunique).reset_index()

course_data_posts['COURSE_PK1'] = course_data_posts['COURSE_PK1'].astype(int)
course_data_posts['CONTEXT_PK1'] = course_data_posts['CONTEXT_PK1'].astype(int)

course_data_posts_consume['COURSE_PK1'] =course_data_posts_consume['COURSE_PK1'].astype(int)
course_data_posts_consume['CONTEXT_PK1'] = course_data_posts_consume['CONTEXT_PK1'].astype(int)

course_data_posts_consume = course_data_posts_consume.merge(pd.pivot_table(course_data_posts, values='POST_PK1', index=['COURSE_PK1', 'CONTEXT_PK1'],
                     aggfunc=pd.Series.nunique).reset_index(), on = ['COURSE_PK1','CONTEXT_PK1'], how = 'left')

course_data_posts_consume['NEW_NUM_READ_POSTS'] = course_data_posts_consume.apply(lambda x: x['NUM_READ_POSTS'] if x['NUM_READ_POSTS'] < x['POST_PK1'] else x['POST_PK1'], axis = 1)

proportion_posts_read = pd.pivot_table(course_data_posts_consume, values='NEW_NUM_READ_POSTS', index=['COURSE_PK1', 'USER_PK1'],
                     aggfunc='sum').reset_index()
proportion_posts_read = proportion_posts_read.merge(total_number_posts, on = ['COURSE_PK1'])

proportion_posts_read['PROPORTION_POSTS_READ'] = proportion_posts_read['NEW_NUM_READ_POSTS']/proportion_posts_read['TOTAL']

plt.hist(proportion_posts_read['PROPORTION_POSTS_READ'], bins= 5)
plt.title('PROPORTION OF POSTS READ')

### Feature N°3: POSTS_CREATED
"The amount of posts a student has created"

In [ ]:
posts_created = pd.pivot_table(course_data_posts, values='POST_PK1', index=['COURSE_PK1', 'USER_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'POST_PK1':'POSTS_CREATED'})

print(posts_created.head())

plt.hist(posts_created['POSTS_CREATED'], bins= 15)
plt.title('POSTS_CREATED')

## Overall Regularity of Study

###  Feature N°1: ENTROPY_CLICKS
"Entropy of clicks/learning action counts per session"

In [ ]:
# calculating total number of clicks per student per course
sum_clicks_dict = pd.pivot_table(clicks_per_session, values='EVENTS', index=['COURSE_PK1', 'USER_PK1'],
                     aggfunc='sum').to_dict()

clicks_per_session['TOTAL_CLICKS'] = clicks_per_session.apply(lambda x: sum_clicks_dict['EVENTS'][(x['COURSE_PK1'], x['USER_PK1'])], axis = 1)

print(clicks_per_session.head())

clicks_per_session['PROBA'] = clicks_per_session['EVENTS']/clicks_per_session['TOTAL_CLICKS']

print(clicks_per_session.head())

In [ ]:
# Entropy of learning action counts per session (session of length of 0 are excluded)
clicks_entropy = pd.DataFrame(clicks_per_session.groupby(by = ['COURSE_PK1', 'USER_PK1'])['PROBA'].apply(lambda x: entropy(x, base=2))).reset_index()

print(clicks_entropy.head())

plt.hist(clicks_entropy['PROBA'], bins= 10)

plt.title('ENTROPY OF LEARNING ACTION COUNTS PER SESSION')

### Feature N°2: ENTROPY_SESSION_LENGTH
"Entropy of students’ session lengths"

In [ ]:
session_duration_user_dict = pd.pivot_table(session_duration, values='SECONDS', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='sum').fillna(0).rename(columns = {'SECONDS':'SESSION_DURATION'}).to_dict()
session_duration['TOTAL_DURATION'] = session_duration.apply(lambda x: session_duration_user_dict['SESSION_DURATION'][(x['COURSE_PK1'], x['USER_PK1'])], axis = 1)
session_duration['PROBA'] = session_duration['SECONDS']/session_duration['TOTAL_DURATION']

# Entropy of session length
duration_entropy = pd.DataFrame(session_duration.groupby(by = ['COURSE_PK1', 'USER_PK1'])['PROBA'].apply(lambda x: entropy(x, base=2))).reset_index()

print(duration_entropy.head())

plt.hist(duration_entropy['PROBA'], bins= 10)

plt.title('ENTROPY OF SESSION LENGTH')

### Feature N°3: PROPORTION_WEEKS_FIRST_DAY_ACTIVE
"The proportion of weeks for which the students were active on monday"

In [ ]:
course_data['DAY_OF_WEEK'] = course_data.apply(lambda x: x['DATE'].isoweekday(), axis = 1)

weeks_with_active_first_day = pd.pivot_table(course_data[course_data['DAY_OF_WEEK'] == 1], values='WEEK and YEAR', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'WEEK and YEAR':'ACTIVE_WEEKS'})
weeks_with_active_first_day
weeks_with_active_first_day['PROPORTION_ACTIVE_WEEKS'] = weeks_with_active_first_day['ACTIVE_WEEKS']/course_duration_weeks

print(weeks_with_active_first_day.head())

plt.hist(weeks_with_active_first_day['PROPORTION_ACTIVE_WEEKS'], bins= 10)

plt.title('PROPORTION OF WEEKS WITH ACTIVE FIRST DAY')

### Feature N°4: PROPORTION_LA_FIRST_DAY_OF_WEEK
"Proportion of learning actions on monday"

In [ ]:
course_data_non_zero['DAY_OF_WEEK'] = course_data_non_zero.apply(lambda x: x['DATE'].isoweekday(), axis = 1)

clicks_per_timestamp = pd.pivot_table(course_data_non_zero, values='EVENTS', index=['COURSE_PK1','USER_PK1', 'new_session_id','TIMESTAMP_NEW'],
                     aggfunc='sum').fillna(0).reset_index()

clicks_per_timestamp.sort_values(by = 'EVENTS', ascending = False)

# replace the rows with #events > 1 by 1
# as Toledo logs all the subfolders of an opened parent folder as a separate event, we replace this multiple events by just 1 event of openening a folder
mask = clicks_per_timestamp['EVENTS'] > 1
column_name = 'EVENTS'
clicks_per_timestamp.loc[mask, column_name] = 1
clicks_per_timestamp.sort_values(by = 'EVENTS', ascending = False)

clicks_per_timestamp['DAY_OF_WEEK'] = clicks_per_timestamp.apply(lambda x: x['TIMESTAMP_NEW'].isoweekday(), axis = 1)

clicks_per_timestamp['WEEK'] = clicks_per_timestamp.apply(lambda x: x['TIMESTAMP_NEW'].isocalendar()[1], axis = 1)
clicks_per_timestamp['YEAR'] = clicks_per_timestamp.apply(lambda x: x['TIMESTAMP_NEW'].isocalendar()[0], axis = 1)
clicks_per_timestamp['WEEK'] = clicks_per_timestamp['WEEK'].astype(str)
clicks_per_timestamp['YEAR'] = clicks_per_timestamp['YEAR'].astype(str)
clicks_per_timestamp['WEEK and YEAR'] = 'Week ' + clicks_per_timestamp['WEEK'] + ' of ' + clicks_per_timestamp['YEAR']
clicks_per_timestamp['DATE'] = clicks_per_timestamp.apply(lambda x: x['TIMESTAMP_NEW'].date(), axis = 1)

total_learning_actions = pd.pivot_table(clicks_per_timestamp, values='EVENTS', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR'],
                     aggfunc='sum').fillna(0).reset_index()

print(total_learning_actions.head())

learning_actions_first_day = pd.pivot_table(clicks_per_timestamp[clicks_per_timestamp['DAY_OF_WEEK'] == 1], values='EVENTS', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR'],
                     aggfunc='sum').fillna(0).reset_index().rename(columns = {'EVENTS':'EVENTS_FIRST_DAY'})

print(learning_actions_first_day.head())

total_learning_actions = total_learning_actions.merge(learning_actions_first_day, on = ['COURSE_PK1','USER_PK1','WEEK and YEAR'], how = 'left').fillna(0)
total_learning_actions['PROPORTION'] = total_learning_actions['EVENTS_FIRST_DAY']/total_learning_actions['EVENTS']

print(total_learning_actions.head())

In [ ]:
average_proportion_la_first_day = pd.pivot_table(total_learning_actions, values='PROPORTION', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='median').fillna(0).reset_index()

print(average_proportion_la_first_day.head())

plt.hist(average_proportion_la_first_day['PROPORTION'], bins= 15)

plt.title('PROPORTION OF LA ON THE FIRST DAY')

## Learning Actions Specific Regularity of Study

### Feature N°1: ENTROPY_FORUM_CONTRIBUTION_DAILY
"Entropy of daily posts written "

In [ ]:
n_weeks = len(set(course_data_non_zero['WEEK and YEAR']))

posts_per_day = pd.pivot_table(course_data_posts, values='POST_PK1', index=['COURSE_PK1','USER_PK1', 'DATE_CREATED'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'POST_PK1':'POSTS_PER_DAY'})

print(posts_per_day.head())

posts_total = pd.pivot_table(course_data_posts, values='POST_PK1', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'POST_PK1':'POSTS_TOTAL'})

print(posts_total.head())

posts_per_day = posts_per_day.merge(posts_total, how = 'left', on = ['COURSE_PK1', 'USER_PK1'])
posts_per_day['PROBA'] = (posts_per_day['POSTS_PER_DAY']/posts_per_day['POSTS_TOTAL']).fillna(0)

posts_per_day = pd.DataFrame(posts_per_day.groupby(by = ['COURSE_PK1', 'USER_PK1'])['PROBA'].apply(lambda x: entropy(x, base=2))).reset_index().fillna(math.log2(course_duration))

print(posts_per_day.head())

plt.hist(posts_per_day['PROBA'], bins= 10)

plt.title('ENTROPY OF DAILY POSTS WRITTEN')

### Feature N°2: ENTROPY_FORUM_CONTRIBUTION_WEEKLY
"Entropy of weekly posts written"

In [ ]:
posts_per_week = pd.pivot_table(course_data_posts, values='POST_PK1', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR created'],
                     aggfunc=pd.Series.nunique).reset_index().rename(columns = {'POST_PK1':'POSTS_PER_WEEK'})

posts_per_week = posts_per_week.merge(posts_total, how = 'left', on = ['COURSE_PK1', 'USER_PK1'])
posts_per_week['PROBA'] = (posts_per_week['POSTS_PER_WEEK']/posts_per_week['POSTS_TOTAL']).fillna(0)

posts_per_week = pd.DataFrame(posts_per_week.groupby(by = ['COURSE_PK1', 'USER_PK1'])['PROBA'].apply(lambda x: entropy(x, base=2))).reset_index().fillna(math.log2(n_weeks))

plt.hist(posts_per_week['PROBA'], bins= 15)

plt.title('ENTROPY OF WEEKLY POSTS WRITTEN')

# MoneyWalks feature engineering

### Feature N°1: DIVERSITY_OVERALL
"A measure that refers  to how evenly sessions are distributed across weeks"

In [ ]:
sessions_per_week = pd.pivot_table(course_data_non_zero, values='new_session_id', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index().rename(columns = {'new_session_id':'SESSIONS_PER_WEEK'})

print(sessions_per_week.head())

sessions_total = pd.pivot_table(course_data_non_zero, values='new_session_id', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index().rename(columns = {'new_session_id':'SESSIONS_TOTAL'})

print(sessions_total.head())

weeks_count_total = pd.pivot_table(course_data_non_zero, values='WEEK and YEAR', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index()
weeks_count_total['LOG NUM OF WEEKS'] = weeks_count_total.apply(lambda x: math.log2(x['WEEK and YEAR']), axis = 1)

print(weeks_count_total.head())

sessions_per_week = sessions_per_week.merge(sessions_total, how = 'left', on = ['COURSE_PK1', 'USER_PK1'])
sessions_per_week['PROBA'] = (sessions_per_week['SESSIONS_PER_WEEK']/sessions_per_week['SESSIONS_TOTAL']).fillna(0)


sessions_per_week_diversity = pd.DataFrame(sessions_per_week.groupby(by = ['COURSE_PK1', 'USER_PK1'])['PROBA'].apply(lambda x: entropy(x, base=2))).reset_index().fillna(math.log2(n_weeks))
sessions_per_week_diversity = sessions_per_week_diversity.merge(weeks_count_total, how = 'left', on = ['COURSE_PK1','USER_PK1'])
sessions_per_week_diversity['DIVERSITY'] =  (sessions_per_week_diversity['PROBA']/sessions_per_week_diversity['LOG NUM OF WEEKS'])

print(sessions_per_week.head())
print(sessions_per_week_diversity.head())

plt.hist(sessions_per_week_diversity['DIVERSITY'], bins= 15)

plt.title('DIVERSITY OVERALL')

### Feature N°2: DIVERSITY_EXAM_PERIOD
"A measure that refers  to how evenly sessions are distributed across weeks during the exam period"

In [ ]:
sessions_per_week_exam = pd.pivot_table(course_data_non_zero[course_data_non_zero['WEEK and YEAR'].isin(exam_weeks)], values='new_session_id', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index().rename(columns = {'new_session_id':'SESSIONS_PER_WEEK_EXAM'})

print(sessions_per_week_exam.head())

In [ ]:
sessions_total_exam = pd.pivot_table(course_data_non_zero[course_data_non_zero['WEEK and YEAR'].isin(exam_weeks)], values='new_session_id', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index().rename(columns = {'new_session_id':'SESSIONS_TOTAL_EXAM'})

print(sessions_total_exam)

In [ ]:
weeks_count_total_exam = pd.pivot_table(course_data_non_zero[course_data_non_zero['WEEK and YEAR'].isin(exam_weeks)], values='WEEK and YEAR', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index()
weeks_count_total_exam['LOG NUM OF WEEKS'] = weeks_count_total_exam.apply(lambda x: math.log2(x['WEEK and YEAR']), axis = 1)

In [ ]:
print(sessions_per_week_exam)
print(sessions_per_week_diversity_exam)

In [ ]:
sessions_per_week_exam = sessions_per_week_exam.merge(sessions_total_exam, how = 'left', on = ['COURSE_PK1', 'USER_PK1'])

sessions_per_week_exam['PROBA'] = (sessions_per_week_exam['SESSIONS_PER_WEEK_EXAM']/sessions_per_week_exam['SESSIONS_TOTAL_EXAM']).fillna(0)


sessions_per_week_diversity_exam = pd.DataFrame(sessions_per_week_exam.groupby(by = ['COURSE_PK1', 'USER_PK1'])['PROBA'].apply(lambda x: entropy(x, base=2))).reset_index().fillna(math.log2(n_weeks))
sessions_per_week_diversity_exam = sessions_per_week_diversity_exam.merge(weeks_count_total_exam, how = 'left', on = ['COURSE_PK1','USER_PK1'])
sessions_per_week_diversity_exam['DIVERSITY_EXAM_PERIOD'] =  (sessions_per_week_diversity_exam['PROBA']/sessions_per_week_diversity_exam['LOG NUM OF WEEKS'])

plt.hist(sessions_per_week_diversity_exam['DIVERSITY_EXAM_PERIOD'], bins= 15)

plt.title('DIVERSITY EXAM PERIOD')

## Loyalty

### Feature N°1: LOYALTY_OVERALL
"Proportion of sessions that took place in the top 3 weeks regarding session count. A higher value represents many sessions concentrated in only a few weeks"

In [ ]:
top3_sessions = sessions_per_week.groupby(by = ['COURSE_PK1','USER_PK1']).apply(lambda x: x.nlargest(3, "SESSIONS_PER_WEEK")).reset_index(drop=True)

sessions_total_in_top3_weeks = pd.pivot_table(top3_sessions, values='SESSIONS_PER_WEEK', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='sum').fillna(0).reset_index().rename(columns = {'SESSIONS_PER_WEEK':'SESSIONS_TOTAL_TOP3'})

sessions_total_in_top3_weeks = sessions_total_in_top3_weeks.merge(sessions_total, how='left', on = ['COURSE_PK1','USER_PK1'])

sessions_total_in_top3_weeks['LOYALTY'] = sessions_total_in_top3_weeks['SESSIONS_TOTAL_TOP3']/sessions_total_in_top3_weeks['SESSIONS_TOTAL']

plt.hist(sessions_total_in_top3_weeks['LOYALTY'], bins= 15)

plt.title('LOYALTY')

### Feature N°2: LOYALTY_EXAM
"Proportion of sessions that took place in the top 3 weeks regarding session count, focusing on exam period"

In [ ]:
exam_sessions = sessions_per_week[sessions_per_week['WEEK and YEAR'].isin(exam_weeks)]

print(exam_sessions.head())

sessions_total_in_exam_weeks = pd.pivot_table(exam_sessions, values='SESSIONS_PER_WEEK', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='sum').fillna(0).reset_index().rename(columns = {'SESSIONS_PER_WEEK':'SESSIONS_TOTAL_EXAM'})

sessions_total_in_exam_weeks = sessions_total_in_exam_weeks.merge(sessions_total, how='left', on = ['COURSE_PK1','USER_PK1'])

sessions_total_in_exam_weeks['LOYALTY'] = sessions_total_in_exam_weeks['SESSIONS_TOTAL_EXAM']/sessions_total_in_exam_weeks['SESSIONS_TOTAL']

plt.hist(sessions_total_in_exam_weeks['LOYALTY'], bins= 15)

plt.title('LOYALTY EXAM PERIOD')

## Regularity

### Feature N°1: REGULARITY
"Measurement for the level of diversity in students’ behavior over time"

### Diversity semester weeks

In [ ]:
course_data_non_zero_semester = course_data_non_zero[course_data_non_zero['WEEK and YEAR'].isin(semester_weeks)]

sessions_per_week_semester = pd.pivot_table(course_data_non_zero_semester, values='new_session_id', index=['COURSE_PK1','USER_PK1', 'WEEK and YEAR'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index().rename(columns = {'new_session_id':'SESSIONS_PER_WEEK'})

sessions_total_semester = pd.pivot_table(course_data_non_zero_semester, values='new_session_id', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index().rename(columns = {'new_session_id':'SESSIONS_TOTAL'})

weeks_count_total_semester = pd.pivot_table(course_data_non_zero_semester, values='WEEK and YEAR', index=['COURSE_PK1','USER_PK1'],
                     aggfunc=pd.Series.nunique).fillna(0).reset_index()
weeks_count_total_semester['LOG NUM OF WEEKS'] = weeks_count_total_semester.apply(lambda x: math.log2(x['WEEK and YEAR']), axis = 1)

sessions_per_week_first_month = sessions_per_week_semester.merge(sessions_total_semester, how = 'left', on = ['COURSE_PK1', 'USER_PK1'])
sessions_per_week_first_month['PROBA'] = (sessions_per_week_first_month['SESSIONS_PER_WEEK']/sessions_per_week_first_month['SESSIONS_TOTAL']).fillna(0)


sessions_per_week_first_month_diversity = pd.DataFrame(sessions_per_week_first_month.groupby(by = ['COURSE_PK1', 'USER_PK1'])['PROBA'].apply(lambda x: entropy(x, base=2))).reset_index().fillna(math.log2(len(semester_weeks)))
sessions_per_week_first_month_diversity = sessions_per_week_first_month_diversity.merge(weeks_count_total_semester, how = 'left', on = ['COURSE_PK1','USER_PK1'])
sessions_per_week_first_month_diversity['DIVERSITY_SEMESTER'] =  sessions_per_week_first_month_diversity['PROBA']/sessions_per_week_first_month_diversity['LOG NUM OF WEEKS']

diversity_for_regularity = sessions_per_week_diversity[['COURSE_PK1','USER_PK1','DIVERSITY']].merge(sessions_per_week_first_month_diversity[['COURSE_PK1','USER_PK1','DIVERSITY_SEMESTER']], how = 'left', on = ['COURSE_PK1','USER_PK1']).fillna(0)

### Loyalty semester weeks

In [ ]:
top3_sessions_semester = sessions_per_week[sessions_per_week['WEEK and YEAR'].isin(semester_weeks)].groupby(by = ['COURSE_PK1','USER_PK1']).apply(lambda x: x.nlargest(3, "SESSIONS_PER_WEEK")).reset_index(drop=True)

sessions_total_in_top3_weeks_semester = pd.pivot_table(top3_sessions_semester, values='SESSIONS_PER_WEEK', index=['COURSE_PK1','USER_PK1'],
                     aggfunc='sum').fillna(0).reset_index().rename(columns = {'SESSIONS_PER_WEEK':'SESSIONS_TOTAL_TOP3'})

sessions_total_in_top3_weeks_semester = sessions_total_in_top3_weeks_semester.merge(sessions_total_semester, how='left', on = ['COURSE_PK1','USER_PK1'])

sessions_total_in_top3_weeks_semester['LOYALTY_SEMESTER'] = sessions_total_in_top3_weeks_semester['SESSIONS_TOTAL_TOP3']/sessions_total_in_top3_weeks_semester['SESSIONS_TOTAL']

regularity_df = sessions_per_week_diversity[['COURSE_PK1','USER_PK1','DIVERSITY']].merge(sessions_total_in_top3_weeks[['COURSE_PK1','USER_PK1','LOYALTY']])

regularity_df = regularity_df.merge(diversity_for_regularity[['COURSE_PK1','USER_PK1','DIVERSITY_SEMESTER']], how = 'left', on = ['COURSE_PK1','USER_PK1']).merge(sessions_total_in_top3_weeks_semester[['COURSE_PK1','USER_PK1','LOYALTY_SEMESTER']], how= 'left', on = ['COURSE_PK1','USER_PK1'] ).fillna(0)

regularity_df['REGULARITY'] = 1- np.sqrt((regularity_df['DIVERSITY_SEMESTER'] - regularity_df['DIVERSITY'] )**2 +  (regularity_df['LOYALTY_SEMESTER'] - regularity_df['LOYALTY'] )**2)/np.sqrt(2)

plt.hist(regularity_df['REGULARITY'], bins= 15)

plt.title('REGULARITY')

# Final merging

In [ ]:
grades['Score juni'] = grades.apply(lambda x: np.nan if (pd.isnull(x['Score juni']) or not str(x['Score juni']).replace('.', '').isdigit()) else int(str(x['Score juni']).split('.')[0]), axis=1)

In [ ]:
# Merging given features
course_grades = grades[['USER_PK1', 'Score januari',
                            'Score juni']].merge(session_count_zero_duration.rename(columns =
                            {'SESSION_COUNT':'ZERO_SESSIONS_COUNT'}),
                             how = 'left', on=['USER_PK1'])

course_grades = course_grades.merge(session_count.rename(columns = {'SESSION_COUNT':'NON_ZERO_SESSION_COUNT'}),
                             how = 'left', on=['COURSE_PK1','USER_PK1'])

course_grades = course_grades.merge(session_duration_user.rename(columns = {'SESSION_DURATION':'TOTAL_SESSION_DURATION'}),
                             how = 'left', on=['COURSE_PK1','USER_PK1'])

course_grades = course_grades.merge(session_duration_user_avg.rename(columns = {'SESSION_DURATION_AVG':'AVERAGE_SESSION_DURATION'}),
                             how = 'left', on=['COURSE_PK1','USER_PK1'])

course_grades = course_grades.merge(clicks_per_user.rename(columns = {'CLICKS_PER_SESSION_AVG':'AVERAGE_ACTIONS_PER_SESSION'}),
                             how = 'left', on=['COURSE_PK1','USER_PK1'])

course_grades = course_grades.merge(num_active_weeks[['COURSE_PK1','USER_PK1','PROPORTION_ACTIVE_WEEKS']],
                             how = 'left', on=['COURSE_PK1','USER_PK1'])

course_grades = course_grades.merge(active_days[['COURSE_PK1','USER_PK1','ACTIVE_DAYS_PROPORTION']],
                             how = 'left', on=['COURSE_PK1','USER_PK1'])

course_grades = course_grades.merge(active_days_per_week_per_user, how = 'left', on=['COURSE_PK1','USER_PK1'])

course_grades = course_grades.merge(avg_time_distance_user.rename(columns = {'AVG_DIFF':'AVG_TIME_DIFF_BETWEEN_ACTIVE_DAYS'}),
                             how = 'left', on=['COURSE_PK1','USER_PK1'])

course_grades['COURSE_PK1'] = course_grades['COURSE_PK1'].astype(float).fillna(0)
course_grades['USER_PK1']=course_grades['USER_PK1'].astype(str)
course_grades['COURSE_PK1']=course_grades['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(active_days_posts_written.rename(columns = {'ACTIVE_DAYS_PROPORTION':
                                'ACTIVE_DAYS_PROPORTION_FORUM_CONTRIBUTION'})[['COURSE_PK1','USER_PK1',
                                'ACTIVE_DAYS_PROPORTION_FORUM_CONTRIBUTION']], how = 'left', on=['COURSE_PK1','USER_PK1'])

course_grades = course_grades.merge(num_active_weeks_posts_written.rename(columns = {'PROPORTION_ACTIVE_WEEKS':
                                'PROPORTION_ACTIVE_WEEKS_FORUM_CONTRIBUTION'})[['COURSE_PK1','USER_PK1',
                                'PROPORTION_ACTIVE_WEEKS_FORUM_CONTRIBUTION']], how = 'left', on=['COURSE_PK1','USER_PK1'])
course_grades['USER_PK1']=course_grades['USER_PK1'].astype(int)

proportion_posts_read['USER_PK1'] = proportion_posts_read['USER_PK1'].astype(int)
course_grades = course_grades.merge(proportion_posts_read[['COURSE_PK1','USER_PK1','PROPORTION_POSTS_READ']],
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])

posts_created['USER_PK1']=posts_created['USER_PK1'].astype(int)
posts_created['COURSE_PK1']=posts_created['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(posts_created[['COURSE_PK1','USER_PK1','POSTS_CREATED']],how = 'left', on=['COURSE_PK1','USER_PK1'])

clicks_entropy['USER_PK1']=clicks_entropy['USER_PK1'].astype(int)
clicks_entropy['COURSE_PK1']=clicks_entropy['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(clicks_entropy.rename(columns={'PROBA':'ENTROPY_CLICKS'}),how = 'left', on=['COURSE_PK1','USER_PK1'])

duration_entropy['USER_PK1']=duration_entropy['USER_PK1'].astype(int)
duration_entropy['COURSE_PK1']=duration_entropy['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(duration_entropy.rename(columns={'PROBA':'ENTROPY_SESSION_LENGTH'}),
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])
weeks_with_active_first_day['USER_PK1']=weeks_with_active_first_day['USER_PK1'].astype(int)
weeks_with_active_first_day['COURSE_PK1']=weeks_with_active_first_day['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(weeks_with_active_first_day.rename(columns={'PROPORTION_ACTIVE_WEEKS':
                                    'PROPORTION_WEEKS_FIRST_DAY_ACTIVE'})[['COURSE_PK1','USER_PK1',
                                    'PROPORTION_WEEKS_FIRST_DAY_ACTIVE']], how = 'left', on=['COURSE_PK1','USER_PK1'])
average_proportion_la_first_day['USER_PK1']=average_proportion_la_first_day['USER_PK1'].astype(int)
average_proportion_la_first_day['COURSE_PK1']=average_proportion_la_first_day['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(average_proportion_la_first_day.rename(columns={'PROPORTION':'PROPORTION_LA_FIRST_DAY_OF_WEEK'}),
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])

posts_per_day['USER_PK1']=posts_per_day['USER_PK1'].astype(int)
posts_per_day['COURSE_PK1']=posts_per_day['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(posts_per_day.rename(columns={'PROBA':'ENTROPY_FORUM_CONTRIBUTION_DAILY'}),
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])
posts_per_week['USER_PK1']=posts_per_week['USER_PK1'].astype(int)
posts_per_week['COURSE_PK1']=posts_per_week['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(posts_per_week.rename(columns={'PROBA':'ENTROPY_FORUM_CONTRIBUTION_WEEKLY'}),
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])

sessions_per_week_diversity['USER_PK1']=sessions_per_week_diversity['USER_PK1'].astype(int)
sessions_per_week_diversity['COURSE_PK1']=sessions_per_week_diversity['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(sessions_per_week_diversity.rename(columns={'DIVERSITY':
                                'DIVERSITY_OVERALL'})[['COURSE_PK1','USER_PK1','DIVERSITY_OVERALL']],
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])
sessions_per_week_diversity_exam['USER_PK1']=sessions_per_week_diversity_exam['USER_PK1'].astype(int)
sessions_per_week_diversity_exam['COURSE_PK1']=sessions_per_week_diversity_exam['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(sessions_per_week_diversity_exam[['COURSE_PK1','USER_PK1','DIVERSITY_EXAM_PERIOD']],
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])

sessions_total_in_top3_weeks['USER_PK1']=sessions_total_in_top3_weeks['USER_PK1'].astype(int)
sessions_total_in_top3_weeks['COURSE_PK1']=sessions_total_in_top3_weeks['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(sessions_total_in_top3_weeks.rename(columns={'LOYALTY':
                                'LOYALTY_OVERALL'})[['COURSE_PK1','USER_PK1','LOYALTY_OVERALL']],
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])
sessions_total_in_exam_weeks['USER_PK1']=sessions_total_in_exam_weeks['USER_PK1'].astype(int)
sessions_total_in_exam_weeks['COURSE_PK1']=sessions_total_in_exam_weeks['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(sessions_total_in_exam_weeks.rename(columns={'LOYALTY':
                                'LOYALTY_EXAM'})[['COURSE_PK1','USER_PK1','LOYALTY_EXAM']],
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])
regularity_df['USER_PK1']=regularity_df['USER_PK1'].astype(int)
regularity_df['COURSE_PK1']=regularity_df['COURSE_PK1'].astype(int)

course_grades = course_grades.merge(regularity_df[['COURSE_PK1','USER_PK1','REGULARITY']],
                                 how = 'left', on=['COURSE_PK1','USER_PK1'])
print(course_grades)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

course_grades['Score juni'] = course_grades['Score juni'].astype(int)
course_grades['USER_PK1'] = course_grades['USER_PK1'].astype(str)

#Merging it all together
merged_data_test = pd.merge(course_grades,workingdata_FI1 , on=["USER_PK1"], how='left')
merged_data_test = pd.merge(merged_data_test, workingdata_FI2, on=["USER_PK1"] , how='left')
merged_data_test = pd.merge(merged_data_test, workingdata_FI3, on=["USER_PK1"], how='left')
merged_data_test = pd.merge(merged_data_test, workingdata_FI4, on=["USER_PK1"], how='left')
merged_data_test = pd.merge(merged_data_test,workingdata_FI5, on=["USER_PK1"], how='left')
merged_data_test = pd.merge(merged_data_test, workingdata_FT2, on=["USER_PK1"], how='left')
merged_data_test = pd.merge(merged_data_test, workingdata_FT5, on=["USER_PK1"], how='left')
merged_data_test = pd.merge(merged_data_test, work, on=["USER_PK1"], how='left')
merged_data_test = pd.merge(merged_data_test, workingdata_FR1, on=["USER_PK1"], how='left')
merged_data_test = pd.merge(merged_data_test, workingdata_FR2, on=["USER_PK1"], how='left')
merged_data_test = pd.merge(merged_data_test, workingdata_FR3, on=["USER_PK1"], how='left')
merged_data_test = pd.merge(merged_data_test, workingdata_FR4, on=["USER_PK1"], how='left')

#Table check
print(merged_data_test)

#Filling NaN
merged_data_test['Score_bins'] = merged_data_test['Score_bins'].astype(str).fillna(0)
merged_data_test.fillna(0, inplace=True)
print(merged_data_test)

In [ ]:
merged_data1 = merged_data1.drop(merged_data_test[merged_data_test['Score juni'] == 0].index)
merged_data_test = merged_data_test.drop(merged_data_test[merged_data_test['Score juni'] == 0].index)


### Defining X_train, Y_train, X_test, Y_test

In [ ]:
#Defining what columns to be used
X_train = merged_data1.drop(columns=[ 'Score januari' , 'Score juni','USER_PK1','COURSE_PK1','Score_bins'])
y_train = merged_data1.drop(columns=['sum_read_posts', 'sum_comments', 'sum_views', 'sum_ind_read', 'mean_text_length','days_until_first_login','entropy_per_student','above_median_count','percentageweeks_above_median', 'aggregate_mean_squared_diff_per_student','total_score_FR2', 'exceptional_view','total_score','ZERO_SESSIONS_COUNT', 'NON_ZERO_SESSION_COUNT', 'TOTAL_SESSION_DURATION',
            'AVERAGE_SESSION_DURATION', 'AVERAGE_ACTIONS_PER_SESSION', 'PROPORTION_ACTIVE_WEEKS',
            'ACTIVE_DAYS_PROPORTION', 'AVG_TIME_DIFF_BETWEEN_ACTIVE_DAYS',
            'ACTIVE_DAYS_PROPORTION_FORUM_CONTRIBUTION', 'PROPORTION_ACTIVE_WEEKS_FORUM_CONTRIBUTION',
            'PROPORTION_POSTS_READ', 'POSTS_CREATED', 'ENTROPY_CLICKS', 'ENTROPY_SESSION_LENGTH',
            'PROPORTION_WEEKS_FIRST_DAY_ACTIVE', 'PROPORTION_LA_FIRST_DAY_OF_WEEK',
            'ENTROPY_FORUM_CONTRIBUTION_DAILY', 'ENTROPY_FORUM_CONTRIBUTION_WEEKLY', 'DIVERSITY_OVERALL',
            'DIVERSITY_EXAM_PERIOD', 'LOYALTY_OVERALL', 'LOYALTY_EXAM', 'REGULARITY','AVERAGE_ACTIVE_DAYS_PER_WEEK','COURSE_PK1', 'Score januari','USER_PK1','Score_bins'])

X_test= merged_data_test.drop(columns=['Score januari',  'Score juni','USER_PK1','COURSE_PK1','Score_bins'])
y_test= merged_data_test.drop(columns=['sum_read_posts', 'sum_comments', 'sum_views', 'sum_ind_read', 'mean_text_length','days_until_first_login','entropy_per_student','above_median_count','percentageweeks_above_median', 'aggregate_mean_squared_diff_per_student','total_score_FR2', 'exceptional_view','total_score','ZERO_SESSIONS_COUNT', 'NON_ZERO_SESSION_COUNT', 'TOTAL_SESSION_DURATION',
            'AVERAGE_SESSION_DURATION', 'AVERAGE_ACTIONS_PER_SESSION', 'PROPORTION_ACTIVE_WEEKS',
            'ACTIVE_DAYS_PROPORTION', 'AVG_TIME_DIFF_BETWEEN_ACTIVE_DAYS',
            'ACTIVE_DAYS_PROPORTION_FORUM_CONTRIBUTION', 'PROPORTION_ACTIVE_WEEKS_FORUM_CONTRIBUTION',
            'PROPORTION_POSTS_READ', 'POSTS_CREATED', 'ENTROPY_CLICKS', 'ENTROPY_SESSION_LENGTH',
            'PROPORTION_WEEKS_FIRST_DAY_ACTIVE', 'PROPORTION_LA_FIRST_DAY_OF_WEEK',
            'ENTROPY_FORUM_CONTRIBUTION_DAILY', 'ENTROPY_FORUM_CONTRIBUTION_WEEKLY', 'DIVERSITY_OVERALL',
            'DIVERSITY_EXAM_PERIOD', 'LOYALTY_OVERALL', 'LOYALTY_EXAM', 'REGULARITY','AVERAGE_ACTIVE_DAYS_PER_WEEK','COURSE_PK1', 'Score januari','USER_PK1','Score_bins'])
print(X_train)
print(y_train)
print(X_test)
print(y_test)

# ------- FEATURE SELECTION -------

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

# Standardizing the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Assuming X_train is your feature matrix and y_train is your target variable
# DataFrame
X = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X.reset_index(drop=True, inplace=True)
y_train = y_train.reset_index(drop=True)

# Performing stepwise selection
def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.30,
                       threshold_out=0.40,
                       verbose=True):
    included = list(initial_list)
    while True:
        changed = False
        # Forward step
        excluded = list(set(X.columns) - set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included + [new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed = True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # Backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # Use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max()  # null if pvalues is empty
        if worst_pval > threshold_out:
            changed = True
            worst_feature = pvalues.argmax()
            worst_feature_name = pvalues.index[worst_feature]  # Get the name of the worst feature
            included.remove(worst_feature_name)  # Remove the feature name instead of its index
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature_name, worst_pval))
        if not changed:
            break

    # Fitting the final model
    final_model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
    # Printing the coefficients
    print("Coefficients of selected features:")
    print(final_model.params)

    # Calculating standard deviations of predictors
    std_devs = X[included].std()
    # Calculating standardized coefficients
    standardized_coefs = final_model.params / std_devs
    # Printing the standardized coefficients
    print("\nStandardized coefficients of selected features:")
    print(standardized_coefs)

    return included

# Performing stepwise selection
selected_features_stepwise = stepwise_selection(X, y_train)

print("\nSelected features using stepwise regression:")
print(selected_features_stepwise)

# Printing the features not selected by stepwise regression
not_selected_features_stepwise = list(set(X.columns) - set(selected_features_stepwise))

print("\nFeatures not selected by stepwise regression:")
print(not_selected_features_stepwise)

Implement results Stepwise Regression

In [ ]:
X_train = X_train.drop(columns= not_selected_features_stepwise)

X_test= X_test.drop(columns= not_selected_features_stepwise)

print(X_train)
print(y_train)
print(X_test)
print(y_test)

# ------- Machine Learning Techniques -------

## Logistic Regression

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, KFold, RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint

# Standardizing the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Defining edges for the bins
bin_edges = [-0.0001, 9, 20]

# Transforming target variables to bins
y_values = y_train['Score juni'].values
y_bins = pd.cut(y_values, bins=bin_edges, labels=False)

# Defining scorers for cross-validation
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average='weighted'),
           'recall': make_scorer(recall_score, average='weighted'),
           'f1': make_scorer(f1_score, average='weighted'),
           'balanced_accuracy': make_scorer(balanced_accuracy_score),
           'specificity': make_scorer(lambda y, y_pred: confusion_matrix(y, y_pred)[0,0] / (confusion_matrix(y, y_pred)[0,0] + confusion_matrix(y, y_pred)[0,1]))}

# Model initialization
model = LogisticRegression(random_state=42)

# Defining the number of folds for inner CV
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Defining hyperparameters with smart parameters
param_dist = {
    'C': randint(1, 100),  # Regularization parameter
    'penalty': ['l1', 'l2'],  # Penalty norm
    'max_iter': randint(100, 1000)  # Maximum number of iterations
}

# Performing nested cross-validation
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
outer_cv_scores = []
test_fold_scores = []
best_params_list = []
for fold_idx, (train_index, test_index) in enumerate(outer_cv.split(X_train_scaled, y_bins), 1):
    X_train_fold, X_test_fold = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_fold, y_test_fold = y_bins[train_index], y_bins[test_index]

    # Performing hyperparameter tuning with inner CV
    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=100, cv=inner_cv,
                                       scoring='accuracy', verbose=2, n_jobs=-1, random_state=42)
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_
    best_params = random_search.best_params_
    best_params_list.append(best_params)


    # Evaluating the best model on the test fold
    y_pred = best_model.predict(X_test_fold)
    accuracy = balanced_accuracy_score(y_test_fold, y_pred)
    outer_cv_scores.append(accuracy)
    test_fold_scores.append(accuracy)

    #Part Given step 1
    print('        Best ACCURACY y1 model %.2f%%' % (random_search.best_score_ * 100))
    print('        Best parameters model:', random_search.best_params_)

    #step 2
    print('        Test ACCURACY  (on outer test fold) %.2f%%' % (outer_cv_scores[-1]*100))

# Printing the average test fold performance
print("\nAverage Test Fold Performance:")
print("Mean Accuracy:", np.mean(test_fold_scores))
print("Std Accuracy:", np.std(test_fold_scores))

# Training the model on the entire training data
model.fit(X_train_scaled, y_bins)

# Predicting test data
predictions_test = model.predict(X_test_scaled)

# Transforming bin edges for test data
y_test_values = y_test['Score juni'].values
y_test_bins = pd.cut(y_test_values, bins=bin_edges, labels=False)

# Evaluating
accuracy_test = accuracy_score(y_test_bins, predictions_test)
precision_test = precision_score(y_test_bins, predictions_test, average='weighted')
recall_test = recall_score(y_test_bins, predictions_test, average='weighted')
f1_test = f1_score(y_test_bins, predictions_test, average='weighted')
balanced_accuracy_test = balanced_accuracy_score(y_test_bins, predictions_test)
specificity_test = confusion_matrix(y_test_bins, predictions_test)[0,0] / (confusion_matrix(y_test_bins, predictions_test)[0,0] + confusion_matrix(y_test_bins, predictions_test)[0,1])

# Printing evaluation metrics
print("\nTest data:")
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("Balanced Accuracy:", balanced_accuracy_test)
print("Specificity:", specificity_test)

# Confusion matrix
conf_matrix_test = confusion_matrix(y_test_bins, predictions_test)
print("\nConfusion Matrix (Test data):")
print(conf_matrix_test)

print(outer_cv_scores)

## Random Forest Classifier

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, KFold, RandomizedSearchCV
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint

# Standardizing the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Defining edges for the bins
bin_edges = [-0.0001, 9, 20]

# Transforming target variables to bins
y_values = y_train['Score juni'].values
y_bins = pd.cut(y_values, bins=bin_edges, labels=False)

# Defining scorers for cross-validation
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average='weighted'),
           'recall': make_scorer(recall_score, average='weighted'),
           'f1': make_scorer(f1_score, average='weighted'),
           'balanced_accuracy': make_scorer(balanced_accuracy_score),
           'specificity': make_scorer(lambda y, y_pred: confusion_matrix(y, y_pred)[0,0] / (confusion_matrix(y, y_pred)[0,0] + confusion_matrix(y, y_pred)[0,1]))}

# Model initialization
model = RandomForestClassifier(random_state=42)

# Defining hyperparameters to tune
param_dist = {
    'n_estimators': randint(300, 600),
    'max_depth': randint(20, 100),
    'min_samples_split': randint(2, 20),
    'min_samples_leaf': randint(1, 10)
}

# Defining the number of folds for inner CV
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Performing nested cross-validation
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
outer_cv_scores = []
test_fold_scores = []
best_params_list = []
for fold_idx, (train_index, test_index) in enumerate(outer_cv.split(X_train_scaled, y_bins), 1):
    X_train_fold, X_test_fold = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_fold, y_test_fold = y_bins[train_index], y_bins[test_index]

    # Performing hyperparameter tuning with inner CV
    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=100, cv=inner_cv, scoring=scoring, refit='accuracy', verbose=2, n_jobs=-1, random_state=42)
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_
    best_params = random_search.best_params_
    best_params_list.append(best_params)


    # Evaluating the best model on the test fold
    y_pred = best_model.predict(X_test_fold)
    accuracy = balanced_accuracy_score(y_test_fold, y_pred)
    outer_cv_scores.append(accuracy)
    test_fold_scores.append(accuracy)

    #Part Given step 1
    print('        Best ACCURACY y1 model %.2f%%' % (random_search.best_score_ * 100))
    print('        Best parameters model:', random_search.best_params_)

    #step 2
    print('        Test ACCURACY  (on outer test fold) %.2f%%' % (outer_cv_scores[-1]*100))

# Printing the average test fold performance
print("\nAverage Test Fold Performance:")
print("Mean Accuracy:", np.mean(test_fold_scores))
print("Std Accuracy:", np.std(test_fold_scores))

# Training the model on the entire training data
model.fit(X_train_scaled, y_bins)

# Predicting test data
predictions_test = model.predict(X_test_scaled)

# Transforming bin edges for test data
y_test_values = y_test['Score juni'].values
y_test_bins = pd.cut(y_test_values, bins=bin_edges, labels=False)

# Evaluating
accuracy_test = accuracy_score(y_test_bins, predictions_test)
precision_test = precision_score(y_test_bins, predictions_test, average='weighted')
recall_test = recall_score(y_test_bins, predictions_test, average='weighted')
f1_test = f1_score(y_test_bins, predictions_test, average='weighted')
balanced_accuracy_test = balanced_accuracy_score(y_test_bins, predictions_test)
specificity_test = confusion_matrix(y_test_bins, predictions_test)[0,0] / (confusion_matrix(y_test_bins, predictions_test)[0,0] + confusion_matrix(y_test_bins, predictions_test)[0,1])

# Printing evaluation metrics
print("\nTest data:")
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("Balanced Accuracy:", balanced_accuracy_test)
print("Specificity:", specificity_test)

# Confusion matrix
conf_matrix_test = confusion_matrix(y_test_bins, predictions_test)
print("\nConfusion Matrix (Test data):")
print(conf_matrix_test)

print(outer_cv_scores)

## XGBoost Classifier

In [ ]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.metrics import balanced_accuracy_score, make_scorer, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from scipy.stats import randint

# Standardizing the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Definig edges for the bins
bin_edges = [-0.0001, 9, 20]

# Transforming target variables to bins
y_values = y_train['Score juni'].values
y_bins = pd.cut(y_values, bins=bin_edges, labels=False)

# Defining scorers for cross-validation
scoring = {'accuracy': make_scorer(accuracy_score),
           'precision': make_scorer(precision_score, average='weighted'),
           'recall': make_scorer(recall_score, average='weighted'),
           'f1': make_scorer(f1_score, average='weighted'),
           'balanced_accuracy': make_scorer(balanced_accuracy_score),
           'specificity': make_scorer(lambda y, y_pred: confusion_matrix(y, y_pred)[0,0] / (confusion_matrix(y, y_pred)[0,0] + confusion_matrix(y, y_pred)[0,1]))}

# Model initialization
model = XGBClassifier(random_state=42)

# Defining the number of folds for inner CV
inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)

# Defining hyperparameters with smart parameters
param_dist = {
    'n_estimators': randint(100, 1000),
    'max_depth': randint(3, 10),
    'learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3],
    'min_child_weight': [1, 5, 10],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4]
}

# Performing nested cross-validation
outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
outer_cv_scores = []
test_fold_scores = []
best_params_list = []
for fold_idx, (train_index, test_index) in enumerate(outer_cv.split(X_train_scaled, y_bins), 1):
    X_train_fold, X_test_fold = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_fold, y_test_fold = y_bins[train_index], y_bins[test_index]

    # Performing hyperparameter tuning with inner CV
    random_search = RandomizedSearchCV(estimator=model, param_distributions=param_dist, n_iter=100, cv=inner_cv,
                                       scoring='accuracy', verbose=2, n_jobs=-1, random_state=42)
    random_search.fit(X_train_fold, y_train_fold)
    best_model = random_search.best_estimator_
    best_params = random_search.best_params_
    best_params_list.append(best_params)

    # Evaluating the best model on the test fold
    y_pred = best_model.predict(X_test_fold)
    accuracy = balanced_accuracy_score(y_test_fold, y_pred)
    #print(f"Fold {fold_idx}: Test Accuracy = {accuracy}")
    outer_cv_scores.append(accuracy)
    test_fold_scores.append(accuracy)

    #Part Given step 1
    print('        Best ACCURACY y1 model %.2f%%' % (random_search.best_score_ * 100))
    print('        Best parameters model:', random_search.best_params_)

    #step 2
    print('        Test ACCURACY  (on outer test fold) %.2f%%' % (outer_cv_scores[-1]*100))

# Printing the average test fold performance
print("\nAverage Test Fold Performance:")
print("Mean Accuracy:", np.mean(test_fold_scores))
print("Std Accuracy:", np.std(test_fold_scores))

# Training the model on the entire training data
model.fit(X_train_scaled, y_bins)

# Predicting test data
predictions_test = model.predict(X_test_scaled)

# Transforming bin edges for test data
y_test_values = y_test['Score juni'].values
y_test_bins = pd.cut(y_test_values, bins=bin_edges, labels=False)

# Evaluating
accuracy_test = accuracy_score(y_test_bins, predictions_test)
precision_test = precision_score(y_test_bins, predictions_test, average='weighted')
recall_test = recall_score(y_test_bins, predictions_test, average='weighted')
f1_test = f1_score(y_test_bins, predictions_test, average='weighted')
balanced_accuracy_test = balanced_accuracy_score(y_test_bins, predictions_test)
specificity_test = confusion_matrix(y_test_bins, predictions_test)[0,0] / (confusion_matrix(y_test_bins, predictions_test)[0,0] + confusion_matrix(y_test_bins, predictions_test)[0,1])

# Printing evaluation metrics
print("\nTest data:")
print("Accuracy:", accuracy_test)
print("Precision:", precision_test)
print("Recall:", recall_test)
print("F1 Score:", f1_test)
print("Balanced Accuracy:", balanced_accuracy_test)
print("Specificity:", specificity_test)

# Confusion matrix
conf_matrix_test = confusion_matrix(y_test_bins, predictions_test)
print("\nConfusion Matrix (Test data):")
print(conf_matrix_test)

print(outer_cv_scores)